In [33]:
# ============================================================
# CHALLENGE 2: p-factor regression using cached EEG files
# (train/val split based on filename)
# ============================================================

import os
import torch
import torch.nn as nn
import torch.nn.functional as F
from torch.utils.data import Dataset, DataLoader
import numpy as np
import pandas as pd
from tqdm import tqdm
from sklearn.metrics import mean_absolute_error, mean_squared_error, r2_score
from braindecode.models import EEGNetv4, ShallowFBCSPNet, EEGConformer

# -------------------------
# Configuration
# -------------------------
CACHE_DIR = "/data5/open_data/HBN/cache_eeg_windows"
BIDS_ROOT = "/data5/open_data/HBN/EEG_BIDS"
DATASETS  = [f"ds0055{i:02d}" for i in range(5, 17)]
DEVICE    = "cuda" if torch.cuda.is_available() else "cpu"
N_CHANS = 129     # 캐시 shape[1]
WIN_SIZE = 1000   # 캐시 shape[2]
SFREQ = 100       # 다운샘플링 주파수 (Hz)


print(f"Device: {DEVICE}")

# ------------------------------------------------------------
# participants.tsv 병합 및 p_factor 로드
# ------------------------------------------------------------
def load_all_participants(bids_root=BIDS_ROOT, datasets=DATASETS):
    dfs = []
    for ds in datasets:
        pfile = os.path.join(bids_root, ds, "participants.tsv")
        if os.path.exists(pfile):
            df = pd.read_csv(pfile, sep="\t")
            df["release"] = ds
            dfs.append(df)
    df_all = pd.concat(dfs, ignore_index=True)
    df_all = df_all.drop_duplicates(subset=["participant_id"])
    df_all = df_all.set_index("participant_id")
    return df_all

meta_df   = load_all_participants()
labels_df = meta_df[["p_factor"]].dropna()

# ------------------------------------------------------------
# 캐시 파일 목록 불러오기
# ------------------------------------------------------------
all_cache_files = [f for f in os.listdir(CACHE_DIR) if f.endswith(".npy")]
print(f"Total cached files: {len(all_cache_files)}")

# 파일 이름에 train/val 포함 여부로 분리
train_files = [f for f in all_cache_files if "_train_" in f]
val_files   = [f for f in all_cache_files if "_val_" in f]

print(f"Train cache files: {len(train_files)}")
print(f"Val cache files:   {len(val_files)}")

# ------------------------------------------------------------
# Dataset 클래스 (이미 생성된 캐시를 불러오기)
# ------------------------------------------------------------
class EEGCachedDataset(Dataset):
    def __init__(self, cache_dir, cache_files, labels_df):
        self.cache_dir = cache_dir
        self.cache_files = cache_files
        self.labels_df = labels_df

        self.samples = []
        for fname in tqdm(cache_files, desc="Preparing samples"):
            subj = fname.split("_")[0]  # e.g., sub-NDARFU916UZY
            if subj not in labels_df.index:
                continue
            y = torch.tensor(labels_df.loc[subj, "p_factor"], dtype=torch.float32)
            self.samples.append((fname, y))

    def __len__(self):
        return len(self.samples)

    def __getitem__(self, idx):
        fname, y = self.samples[idx]
        path = os.path.join(self.cache_dir, fname)
        X_all = np.load(path, allow_pickle=True)  # shape: (segments, chans, times)
        X_mean = torch.tensor(X_all.mean(axis=0), dtype=torch.float32)
        return X_mean, y

# ------------------------------------------------------------
# DataLoader 구성
# ------------------------------------------------------------
train_ds = EEGCachedDataset(CACHE_DIR, train_files, labels_df)
val_ds   = EEGCachedDataset(CACHE_DIR, val_files, labels_df)

train_loader = DataLoader(train_ds, batch_size=8, shuffle=True, num_workers=4, pin_memory=True)
val_loader   = DataLoader(val_ds, batch_size=8, shuffle=False, num_workers=4, pin_memory=True)

# ------------------------------------------------------------
# 모델 정의
# ------------------------------------------------------------
def get_model(model_name, n_chans=N_CHANS, n_times=WIN_SIZE, sfreq=100):
    if model_name == "EEGNet":
        m = EEGNetv4(
            n_chans=n_chans,
            n_outputs=1,
            n_times=n_times,
            sfreq=sfreq,
            final_conv_length="auto"
        )
        # m.classifier[-1] = nn.Linear(m.classifier[-1].in_features, 1)

    elif model_name == "ShallowFBCSPNet":
        m = ShallowFBCSPNet(
            n_chans=n_chans,
            n_outputs=1,
            n_times=n_times,
            sfreq=sfreq,
            final_conv_length=30
        )
        # m.regressor = nn.Linear(m.classifier.in_features, 1)

    elif model_name == "EEGConformer":
        m = EEGConformer(
            n_chans=n_chans,
            n_outputs=1,
            n_times=n_times,
            sfreq=sfreq
        )

    else:
        raise ValueError(f"Unknown model: {model_name}")

    return m



# ------------------------------------------------------------
# 학습 함수
# ------------------------------------------------------------
def train_model(model, train_loader, val_loader, model_name, n_epochs=30, lr=1e-4):
    model = model.to(DEVICE)
    opt = torch.optim.AdamW(model.parameters(), lr=lr, weight_decay=1e-3)
    best_mae, best_mse, best_r2 = float("inf"), float("inf"), -1.0

    for ep in range(1, n_epochs + 1):
        # Training
        model.train()
        train_mae, train_mse = 0.0, 0.0
        for X, y in tqdm(train_loader, desc=f"[{model_name}] Epoch {ep}/{n_epochs} (train)", leave=False):
            X, y = X.to(DEVICE), y.to(DEVICE)
            pred = model(X)

            # (B, 1, T) → (B, T)
            if pred.ndim == 3:
                pred = pred.mean(dim=-1)  # time dimension 평균
            if pred.ndim == 2:
                pred = pred.mean(dim=-1)  # (B,1) → (B)

            pred = pred.view(-1)

            loss_mae = F.l1_loss(pred, y)
            loss_mse = F.mse_loss(pred, y)
            loss = 0.5 * (loss_mae + loss_mse)
            opt.zero_grad(); loss.backward(); opt.step()
            train_mae += loss_mae.item(); train_mse += loss_mse.item()
        avg_mae = train_mae / len(train_loader)
        avg_mse = train_mse / len(train_loader)

        # Validation
        model.eval(); preds, trues = [], []
        with torch.no_grad():
            for X, y in tqdm(val_loader, desc=f"[{model_name}] Epoch {ep}/{n_epochs} (val)", leave=False):
                X, y = X.to(DEVICE), y.to(DEVICE)
                p = model(X)
                # (B, 1, T) → (B, T)
                if p.ndim == 3:
                    p = p.mean(dim=-1)  # time dimension 평균
                if p.ndim == 2:
                    p = p.mean(dim=-1)  # (B,1) → (B)
                
                preds.extend(p.cpu().numpy())
                trues.extend(y.cpu().numpy())
        preds, trues = np.array(preds), np.array(trues)
        val_mae = mean_absolute_error(trues, preds)
        val_mse = mean_squared_error(trues, preds)
        val_r2  = r2_score(trues, preds)

        print(f"[{model_name}] Epoch {ep:02d} | Train MAE={avg_mae:.4f}, MSE={avg_mse:.4f} | "
              f"Val MAE={val_mae:.4f}, MSE={val_mse:.4f}, R²={val_r2:.3f}")
        if val_mae < best_mae:
            best_mae, best_mse, best_r2 = val_mae, val_mse, val_r2
            torch.save(model.state_dict(), f"best_{model_name}.pth")

    print(f"\n[{model_name}] Best Val -> MAE={best_mae:.4f}, MSE={best_mse:.4f}, R²={best_r2:.3f}\n")
    return best_mae, best_mse, best_r2

# ------------------------------------------------------------
# 모델 학습
# ------------------------------------------------------------
models = ["EEGNet","ShallowFBCSPNet", "EEGConformer"]
results = {}
for name in models:
    model = get_model(name)
    mae, mse, r2 = train_model(model, train_loader, val_loader, name)
    results[name] = {"MAE": mae, "MSE": mse, "R²": r2}

print("\n=== Model Comparison ===")
for m, v in results.items():
    print(f"{m:20s} | MAE={v['MAE']:.4f} | MSE={v['MSE']:.4f} | R²={v['R²']:.4f}")


Device: cuda
Total cached files: 198
Train cache files: 190
Val cache files:   8


Preparing samples: 100%|██████████| 8/8 [00:00<00:00, 19043.38it/s]
<decorator-gen-493>:4: FutureWarning: NOTE: EEGNetv4() is a deprecated class. `EEGNetv4` was renamed to `EEGNet` in v1.12; this alias will be removed in v1.14..


[EEGNet] Epoch 01 | Train MAE=0.7314, MSE=0.8455 | Val MAE=0.8021, MSE=1.0971, R²=0.093


[EEGNet] Epoch 02 | Train MAE=0.7429, MSE=0.8649 | Val MAE=0.8058, MSE=1.1007, R²=0.090


[EEGNet] Epoch 03 | Train MAE=0.7077, MSE=0.7985 | Val MAE=0.8068, MSE=1.1043, R²=0.087


...

[EEGConformer] Epoch 26 | Train MAE=0.5490, MSE=0.5381 | Val MAE=1.0455, MSE=1.6053, R²=-0.328


[EEGConformer] Epoch 27 | Train MAE=0.5115, MSE=0.4393 | Val MAE=1.1491, MSE=1.7564, R²=-0.453


[EEGConformer] Epoch 28 | Train MAE=0.5636, MSE=0.5385 | Val MAE=1.1200, MSE=1.5770, R²=-0.304


[EEGConformer] Epoch 29 | Train MAE=0.5627, MSE=0.5549 | Val MAE=1.0512, MSE=1.3310, R²=-0.101


[EEGConformer] Epoch 30 | Train MAE=0.5401, MSE=0.4867 | Val MAE=1.1110, MSE=1.4990, R²=-0.240

[EEGConformer] Best Val -> MAE=0.7806, MSE=0.9259, R²=0.234


=== Model Comparison ===
EEGNet               | MAE=0.7501 | MSE=1.0293 | R²=0.1487
ShallowFBCSPNet      | MAE=0.7361 | MSE=0.7930 | R²=0.3442
EEGConformer         | MAE=0.7806 | MSE=0.9259 | R²=0.2343


## EEG Conformer

In [48]:
# ============================================================
# CHALLENGE 2: p-factor regression (EEGConformer backbone)
# Subject-level K-window anchoring + metadata conditioning
# 5-FOLD CV + release-5 validation
# ============================================================

import os, gc, random
import numpy as np
import pandas as pd
from collections import defaultdict

import torch
import torch.nn as nn
import torch.nn.functional as F
from torch.utils.data import Dataset, DataLoader
from tqdm import tqdm
from sklearn.metrics import mean_absolute_error, mean_squared_error, r2_score
from sklearn.model_selection import KFold

from braindecode.models import EEGConformer

# -----------------------------
# CONFIGURATION
# -----------------------------
CACHE_DIR = "/data5/open_data/HBN/cache_eeg_windows"
BIDS_ROOT = "/data5/open_data/HBN/EEG_BIDS"
DATASETS  = [f"ds0055{i:02d}" for i in range(5, 17)]

DEVICE   = "cuda" if torch.cuda.is_available() else "cpu"
SFREQ    = 100
N_CHANS  = 64
CROP_T   = 800
K_WINDOWS = 4

BATCH_TRAIN = 8
EPOCHS = 30
LR = 2e-4
WD = 1e-3
SEED = 42

def set_seed(seed=SEED):
    random.seed(seed); np.random.seed(seed)
    torch.manual_seed(seed); torch.cuda.manual_seed_all(seed)
    torch.backends.cudnn.benchmark = True
set_seed()

print(f"Device: {DEVICE}")

# ============================================================
# 1. PARTICIPANTS & LABELS
# ============================================================
def load_all_participants(bids_root=BIDS_ROOT, datasets=DATASETS):
    dfs = []
    for ds in datasets:
        pfile = os.path.join(bids_root, ds, "participants.tsv")
        if os.path.exists(pfile):
            df = pd.read_csv(pfile, sep="\t")
            df["release"] = ds
            dfs.append(df)
    df_all = pd.concat(dfs, ignore_index=True)
    df_all.columns = [c.lower() for c in df_all.columns]
    df_all = df_all.drop_duplicates(subset=["participant_id"]).set_index("participant_id")
    if "p_factor" not in df_all.columns:
        alt = [c for c in df_all.columns if "p" in c and "factor" in c]
        if alt: df_all = df_all.rename(columns={alt[0]:"p_factor"})
    for col in ["age","sex","ehq_total"]:
        if col not in df_all.columns:
            df_all[col] = np.nan
    return df_all

meta_df = load_all_participants()
labels_df = meta_df[["p_factor"]].dropna()
meta_extra = meta_df[["age","sex","ehq_total"]].copy()

# sex → numeric, 나머지는 표준화
meta_extra["sex"] = meta_extra["sex"].map({"M":1,"F":0,"m":1,"f":0}).fillna(0.5)
for col in ["age","ehq_total"]:
    meta_extra[col] = meta_extra[col].fillna(meta_extra[col].mean())
    m,s = meta_extra[col].mean(), meta_extra[col].std()+1e-6
    meta_extra[col] = (meta_extra[col]-m)/s

print(f"Subjects with p_factor: {len(labels_df)}")

# ============================================================
# 2. CACHED FILES
# ============================================================
all_cache = [f for f in os.listdir(CACHE_DIR) if f.endswith(".npy")]
train_files = [f for f in all_cache if "_train_" in f]
val_files   = [f for f in all_cache if "_val_" in f]   # release-5
print(f"Train cache: {len(train_files)}, Val cache: {len(val_files)}")

def subj_from_fname(fname): return fname.split("_")[0]

def build_window_index(cache_dir, cache_files, label_index):
    idx=[]
    for fname in tqdm(cache_files, desc="Indexing windows"):
        subj=subj_from_fname(fname)
        if subj not in label_index: continue
        arr=np.load(os.path.join(cache_dir,fname),mmap_mode="r")
        for s in range(arr.shape[0]): idx.append((fname,s,subj))
        del arr
    return idx

train_index = build_window_index(CACHE_DIR, train_files, labels_df.index)
val_index   = build_window_index(CACHE_DIR, val_files,   labels_df.index)

def subj_to_indices(index_list):
    d=defaultdict(list)
    for i,(_,_,subj) in enumerate(index_list): d[subj].append(i)
    return d

train_subj_to_idx=subj_to_indices(train_index)
val_subj_to_idx=subj_to_indices(val_index)

print(f"Train subjects: {len(train_subj_to_idx)}, Val subjects: {len(val_subj_to_idx)}")

# ============================================================
# 3. DATASET DEFINITIONS
# ============================================================
def zscore(x,axis=-1,eps=1e-6):
    m=x.mean(axis=axis,keepdims=True); s=x.std(axis=axis,keepdims=True)
    return (x-m)/(s+eps)

class WindowDataset(Dataset):
    def __init__(self, cache_dir, index, labels, crop_len=800, train=True):
        self.cache_dir = cache_dir
        self.index = index
        self.labels = labels
        self.crop_len = crop_len
        self.train = train

        # ✅ 추가: 사용할 채널 인덱스 (앞의 64채널)
        self.sel_ch = np.arange(64)

    def __getitem__(self, i):
        fname, sidx, subj = self.index[i]
        y = float(self.labels.loc[subj])

        x = np.load(os.path.join(self.cache_dir, fname), mmap_mode="r")[sidx]  # (129, T)
        C, T = x.shape

        # ✅ 추가: 64채널만 선택
        if C >= 64:
            x = x[self.sel_ch, :]   # (64, T)

        # crop
        if T > self.crop_len:
            start = np.random.randint(0, T - self.crop_len + 1) if self.train else (T - self.crop_len) // 2
            x = x[:, start:start + self.crop_len]

        x = (x - x.mean(axis=1, keepdims=True)) / (x.std(axis=1, keepdims=True) + 1e-6)
        x = torch.tensor(x).float()  # (64, T)
        return x, torch.tensor(y), subj


class SubjectDataset(Dataset):
    def __init__(self, base_ds, subj_to_idx, meta, K=4, train=True):
        self.base_ds=base_ds; self.subj_to_idx=subj_to_idx
        self.subjs=list(subj_to_idx.keys()); self.meta=meta
        self.K=K; self.train=train
    def __len__(self): return len(self.subjs)
    def __getitem__(self, i):
        subj = self.subjs[i]
        idxs = self.subj_to_idx[subj]

        # ✅ (핵심 수정) — validation에서도 window 개수를 K로 고정
        if self.train:
            # 학습 시: 중복 허용 랜덤 샘플링
            sel = random.choices(idxs, k=self.K)
        else:
            # 검증 시: K개만 샘플 (window 수 부족하면 중복 허용)
            if len(idxs) >= self.K:
                sel = random.sample(idxs, self.K)
            else:
                sel = random.choices(idxs, k=self.K)

        Xs, ys = [], []
        for j in sel:
            x, y, s = self.base_ds[j]
            Xs.append(x)   # x: (C,T)
            ys.append(y)

        X = torch.stack(Xs, dim=0)  # (K, C, T)
        meta_vec = torch.tensor(self.meta.loc[subj].values, dtype=torch.float32)
        y = ys[0]  # 동일 subject → 같은 label
        return X, y, meta_vec, subj

# ============================================================
# 4. MODEL COMPONENTS
# ============================================================
def find_last_linear_and_parent(model):
    target=None; parent=None
    for n,m in model.named_modules():
        if isinstance(m,nn.Linear): target=n
    if target:
        parent=model
        parts=target.split(".")
        for p in parts[:-1]: parent=getattr(parent,p)
        return parts[-1],parent
    return None,None

def get_feat_dim(model):
    x=torch.zeros((2,N_CHANS,CROP_T),device=DEVICE) ###
    with torch.no_grad():
        y=model(x); y=y.view(y.size(0),-1)
    return y.size(1)

class SubjectHead(nn.Module):
    def __init__(self,feat_dim,meta_dim=3,heads=4):
        super().__init__()
        self.attn=nn.MultiheadAttention(feat_dim,heads,batch_first=True)
        self.meta_fc=nn.Sequential(nn.Linear(meta_dim,32),nn.ReLU())
        self.regressor=nn.Sequential(nn.Linear(feat_dim+32,128),nn.ReLU(),nn.Linear(128,1))
    def forward(self,f,meta):
        a,_=self.attn(f,f,f); subj=a.mean(1)
        m=self.meta_fc(meta)
        out=self.regressor(torch.cat([subj,m],1)).squeeze(-1)
        return out

def forward_feats(backbone,X):
    B,K=X.shape[:2]
    X=X.view(B*K,N_CHANS,CROP_T)
    f=backbone(X).view(B*K,-1) ###
    return f.view(B,K,-1)

# ============================================================
# 5. CROSS-VALIDATION
# ============================================================
subjects=np.array(sorted(train_subj_to_idx.keys()))
kf=KFold(n_splits=5,shuffle=True,random_state=42)
folds=list(kf.split(subjects))

y_mean,y_std=labels_df["p_factor"].mean(),labels_df["p_factor"].std()+1e-6
def norm_y(y): return (y-y_mean)/y_std
def denorm_y(y): return y*y_std+y_mean

def evaluate(backbone,head,loader):
    backbone.eval(); head.eval()
    preds,trues=[],[]
    with torch.no_grad():
        for X,y,meta,subj in loader:
            X,meta=X.to(DEVICE),meta.to(DEVICE)
            f=forward_feats(backbone,X)
            p=head(f,meta)
            preds.extend(denorm_y(p.cpu().numpy())); trues.extend(y.numpy())
    return mean_absolute_error(trues,preds),mean_squared_error(trues,preds),r2_score(trues,preds)

for fold,(tr_idx,va_idx) in enumerate(folds,1):
    train_subjs,valid_subjs=subjects[tr_idx],subjects[va_idx]
    tr_index=[t for t in train_index if t[2] in train_subjs]
    va_index=[t for t in train_index if t[2] in valid_subjs]

    tr_base=WindowDataset(CACHE_DIR,tr_index,labels_df["p_factor"],CROP_T,True)
    va_base=WindowDataset(CACHE_DIR,va_index,labels_df["p_factor"],CROP_T,False)
    tr_ds=SubjectDataset(tr_base,subj_to_indices(tr_index),meta_extra,K_WINDOWS,True)
    va_ds=SubjectDataset(va_base,subj_to_indices(va_index),meta_extra,K_WINDOWS,False)

    tr_loader=DataLoader(tr_ds,batch_size=BATCH_TRAIN,shuffle=True,num_workers=4)
    va_loader=DataLoader(va_ds,batch_size=BATCH_TRAIN,shuffle=False,num_workers=2)

    backbone=EEGConformer(n_chans=N_CHANS,n_outputs=1,n_times=CROP_T,sfreq=SFREQ).to(DEVICE)
    name,par=find_last_linear_and_parent(backbone); setattr(par,name,nn.Identity())
    feat_dim=get_feat_dim(backbone)
    head=SubjectHead(feat_dim,3,4).to(DEVICE)
    opt=torch.optim.AdamW(list(backbone.parameters())+list(head.parameters()),lr=LR,weight_decay=WD)
    sch=torch.optim.lr_scheduler.CosineAnnealingLR(opt,T_max=EPOCHS)

    best={"mae":1e9,"mse":1e9,"r2":-1e9}
    for ep in range(1,EPOCHS+1):
        backbone.train(); head.train()
        losses=[]
        for X,y,meta,subj in tr_loader:
            X,meta=X.to(DEVICE),meta.to(DEVICE)
            y_n=torch.tensor(norm_y(y.numpy()),dtype=torch.float32,device=DEVICE)
            opt.zero_grad()
            f=forward_feats(backbone,X)
            p=head(f,meta)
            loss=0.5*F.l1_loss(p,y_n)+0.5*F.mse_loss(p,y_n)
            loss.backward(); opt.step(); losses.append(loss.item())
        sch.step()
        val_mae,val_mse,val_r2=evaluate(backbone,head,va_loader)
        print(f"[Fold{fold}] Ep{ep:02d} | MAE={val_mae:.4f} | R²={val_r2:.3f}")
        if val_mae<best["mae"]:
            best={"mae":val_mae,"mse":val_mse,"r2":val_r2}
            torch.save({"backbone":backbone.state_dict(),"head":head.state_dict()},
                       f"fold{fold}_best.pth")
    print(f"Fold{fold} BEST: MAE={best['mae']:.4f}, R²={best['r2']:.3f}")

# ============================================================
# 6. FINAL RELEASE-5 VALIDATION
# ============================================================
print("\n=== FINAL release-5 VAL SET EVALUATION ===")
val_base=WindowDataset(CACHE_DIR,val_index,labels_df["p_factor"],CROP_T,False)
val_ds=SubjectDataset(val_base,val_subj_to_idx,meta_extra,K_WINDOWS,False)
val_loader=DataLoader(val_ds,batch_size=BATCH_TRAIN,shuffle=False,num_workers=2)

fold_scores=[]
for fold in range(1,6):
    ckpt=torch.load(f"fold{fold}_best.pth",map_location=DEVICE)
    backbone=EEGConformer(n_chans=N_CHANS,n_outputs=1,n_times=CROP_T,sfreq=SFREQ).to(DEVICE)
    name,par=find_last_linear_and_parent(backbone); setattr(par,name,nn.Identity())
    feat_dim=get_feat_dim(backbone)
    head=SubjectHead(feat_dim,3,4).to(DEVICE)
    backbone.load_state_dict(ckpt["backbone"],strict=False)
    head.load_state_dict(ckpt["head"],strict=False)
    mae,mse,r2=evaluate(backbone,head,val_loader)
    fold_scores.append([mae,mse,r2])
    print(f"[Fold{fold}] release-5 VAL: MAE={mae:.4f}, R²={r2:.3f}")

mean_scores=np.mean(fold_scores,axis=0)
print(f"\nAverage release-5 performance: MAE={mean_scores[0]:.4f}, MSE={mean_scores[1]:.4f}, R²={mean_scores[2]:.3f}")


Device: cuda
Subjects with p_factor: 3116
Train cache: 309, Val cache: 8


Indexing windows: 100%|██████████| 8/8 [00:00<00:00, 545.49it/s]


Train subjects: 298, Val subjects: 8
[Fold1] Ep01 | MAE=0.7226 | R²=-0.058
[Fold1] Ep02 | MAE=0.7204 | R²=-0.038
[Fold1] Ep03 | MAE=0.7223 | R²=-0.074
[Fold1] Ep04 | MAE=0.7212 | R²=-0.046
[Fold1] Ep05 | MAE=0.7205 | R²=-0.056
[Fold1] Ep06 | MAE=0.7266 | R²=-0.089
[Fold1] Ep07 | MAE=0.7376 | R²=-0.122
[Fold1] Ep08 | MAE=0.7243 | R²=-0.052
[Fold1] Ep09 | MAE=0.7485 | R²=-0.155
[Fold1] Ep10 | MAE=0.7314 | R²=-0.080
[Fold1] Ep11 | MAE=0.7355 | R²=-0.088
[Fold1] Ep12 | MAE=0.7300 | R²=-0.135
[Fold1] Ep13 | MAE=0.7432 | R²=-0.106
[Fold1] Ep14 | MAE=0.7366 | R²=-0.115
[Fold1] Ep15 | MAE=0.7402 | R²=-0.095
[Fold1] Ep16 | MAE=0.7754 | R²=-0.233
[Fold1] Ep17 | MAE=0.7308 | R²=-0.137
[Fold1] Ep18 | MAE=0.7443 | R²=-0.161
[Fold1] Ep19 | MAE=0.7743 | R²=-0.225
[Fold1] Ep20 | MAE=0.7308 | R²=-0.134
[Fold1] Ep21 | MAE=0.7762 | R²=-0.300
[Fold1] Ep22 | MAE=0.7650 | R²=-0.210
[Fold1] Ep23 | MAE=0.7652 | R²=-0.228
[Fold1] Ep24 | MAE=0.8268 | R²=-0.419
[Fold1] Ep25 | MAE=0.7811 | R²=-0.304
[Fold1] Ep26 

### LOSO CV

In [52]:
# ============================================================
# CHALLENGE 2: p-factor regression (EEGConformer backbone)
# Subject-level K-window anchoring + metadata conditioning
# 5-FOLD CV + release-5 validation
# ============================================================

import os, gc, random
import numpy as np
import pandas as pd
from collections import defaultdict

import torch
import torch.nn as nn
import torch.nn.functional as F
from torch.utils.data import Dataset, DataLoader
from tqdm import tqdm
from sklearn.metrics import mean_absolute_error, mean_squared_error, r2_score
from sklearn.model_selection import KFold

from braindecode.models import EEGConformer

# -----------------------------
# CONFIGURATION
# -----------------------------
CACHE_DIR = "/data5/open_data/HBN/cache_eeg_windows"
BIDS_ROOT = "/data5/open_data/HBN/EEG_BIDS"
DATASETS  = [f"ds0055{i:02d}" for i in range(5, 17)]

DEVICE   = "cuda" if torch.cuda.is_available() else "cpu"
SFREQ    = 100
N_CHANS  = 64
CROP_T   = 800
K_WINDOWS = 4

BATCH_TRAIN = 8
EPOCHS = 10
LR = 2e-4
WD = 1e-3
SEED = 42

def set_seed(seed=SEED):
    random.seed(seed); np.random.seed(seed)
    torch.manual_seed(seed); torch.cuda.manual_seed_all(seed)
    torch.backends.cudnn.benchmark = True
set_seed()

print(f"Device: {DEVICE}")

# ============================================================
# 1. PARTICIPANTS & LABELS
# ============================================================
def load_all_participants(bids_root=BIDS_ROOT, datasets=DATASETS):
    dfs = []
    for ds in datasets:
        pfile = os.path.join(bids_root, ds, "participants.tsv")
        if os.path.exists(pfile):
            df = pd.read_csv(pfile, sep="\t")
            df["release"] = ds
            dfs.append(df)
    df_all = pd.concat(dfs, ignore_index=True)
    df_all.columns = [c.lower() for c in df_all.columns]
    df_all = df_all.drop_duplicates(subset=["participant_id"]).set_index("participant_id")
    if "p_factor" not in df_all.columns:
        alt = [c for c in df_all.columns if "p" in c and "factor" in c]
        if alt: df_all = df_all.rename(columns={alt[0]:"p_factor"})
    for col in ["age","sex","ehq_total"]:
        if col not in df_all.columns:
            df_all[col] = np.nan
    return df_all

meta_df = load_all_participants()
labels_df = meta_df[["p_factor"]].dropna()
meta_extra = meta_df[["age","sex","ehq_total"]].copy()

# sex → numeric, 나머지는 표준화
meta_extra["sex"] = meta_extra["sex"].map({"M":1,"F":0,"m":1,"f":0}).fillna(0.5)
for col in ["age","ehq_total"]:
    meta_extra[col] = meta_extra[col].fillna(meta_extra[col].mean())
    m,s = meta_extra[col].mean(), meta_extra[col].std()+1e-6
    meta_extra[col] = (meta_extra[col]-m)/s

print(f"Subjects with p_factor: {len(labels_df)}")

# ============================================================
# 2. CACHED FILES
# ============================================================
all_cache = [f for f in os.listdir(CACHE_DIR) if f.endswith(".npy")]
train_files = [f for f in all_cache if "_train_" in f]
val_files   = [f for f in all_cache if "_val_" in f]   # release-5
print(f"Train cache: {len(train_files)}, Val cache: {len(val_files)}")

def subj_from_fname(fname): return fname.split("_")[0]

def build_window_index(cache_dir, cache_files, label_index):
    idx=[]
    for fname in tqdm(cache_files, desc="Indexing windows"):
        subj=subj_from_fname(fname)
        if subj not in label_index: continue
        arr=np.load(os.path.join(cache_dir,fname),mmap_mode="r")
        for s in range(arr.shape[0]): idx.append((fname,s,subj))
        del arr
    return idx

train_index = build_window_index(CACHE_DIR, train_files, labels_df.index)
val_index   = build_window_index(CACHE_DIR, val_files,   labels_df.index)

def subj_to_indices(index_list):
    d=defaultdict(list)
    for i,(_,_,subj) in enumerate(index_list): d[subj].append(i)
    return d

train_subj_to_idx=subj_to_indices(train_index)
val_subj_to_idx=subj_to_indices(val_index)

print(f"Train subjects: {len(train_subj_to_idx)}, Val subjects: {len(val_subj_to_idx)}")

# ============================================================
# 3. DATASET DEFINITIONS
# ============================================================
def zscore(x,axis=-1,eps=1e-6):
    m=x.mean(axis=axis,keepdims=True); s=x.std(axis=axis,keepdims=True)
    return (x-m)/(s+eps)

class WindowDataset(Dataset):
    def __init__(self, cache_dir, index, labels, crop_len=800, train=True):
        self.cache_dir = cache_dir
        self.index = index
        self.labels = labels
        self.crop_len = crop_len
        self.train = train

        # ✅ 추가: 사용할 채널 인덱스 (앞의 64채널)
        self.sel_ch = np.arange(64)

    def __getitem__(self, i):
        fname, sidx, subj = self.index[i]
        y = float(self.labels.loc[subj])

        x = np.load(os.path.join(self.cache_dir, fname), mmap_mode="r")[sidx]  # (129, T)
        C, T = x.shape

        # ✅ 추가: 64채널만 선택
        if C >= 64:
            x = x[self.sel_ch, :]   # (64, T)

        # crop
        if T > self.crop_len:
            start = np.random.randint(0, T - self.crop_len + 1) if self.train else (T - self.crop_len) // 2
            x = x[:, start:start + self.crop_len]

        x = (x - x.mean(axis=1, keepdims=True)) / (x.std(axis=1, keepdims=True) + 1e-6)
        x = torch.tensor(x).float()  # (64, T)
        return x, torch.tensor(y), subj


class SubjectDataset(Dataset):
    def __init__(self, base_ds, subj_to_idx, meta, K=4, train=True):
        self.base_ds=base_ds; self.subj_to_idx=subj_to_idx
        self.subjs=list(subj_to_idx.keys()); self.meta=meta
        self.K=K; self.train=train
    def __len__(self): return len(self.subjs)
    def __getitem__(self, i):
        subj = self.subjs[i]
        idxs = self.subj_to_idx[subj]

        # ✅ (핵심 수정) — validation에서도 window 개수를 K로 고정
        if self.train:
            # 학습 시: 중복 허용 랜덤 샘플링
            sel = random.choices(idxs, k=self.K)
        else:
            # 검증 시: K개만 샘플 (window 수 부족하면 중복 허용)
            if len(idxs) >= self.K:
                sel = random.sample(idxs, self.K)
            else:
                sel = random.choices(idxs, k=self.K)

        Xs, ys = [], []
        for j in sel:
            x, y, s = self.base_ds[j]
            Xs.append(x)   # x: (C,T)
            ys.append(y)

        X = torch.stack(Xs, dim=0)  # (K, C, T)
        meta_vec = torch.tensor(self.meta.loc[subj].values, dtype=torch.float32)
        y = ys[0]  # 동일 subject → 같은 label
        return X, y, meta_vec, subj

# ============================================================
# 4. MODEL COMPONENTS
# ============================================================
def find_last_linear_and_parent(model):
    target=None; parent=None
    for n,m in model.named_modules():
        if isinstance(m,nn.Linear): target=n
    if target:
        parent=model
        parts=target.split(".")
        for p in parts[:-1]: parent=getattr(parent,p)
        return parts[-1],parent
    return None,None

def get_feat_dim(model):
    x=torch.zeros((2,N_CHANS,CROP_T),device=DEVICE) ###
    with torch.no_grad():
        y=model(x); y=y.view(y.size(0),-1)
    return y.size(1)

class SubjectHead(nn.Module):
    def __init__(self,feat_dim,meta_dim=3,heads=4):
        super().__init__()
        self.attn=nn.MultiheadAttention(feat_dim,heads,batch_first=True)
        self.meta_fc=nn.Sequential(nn.Linear(meta_dim,32),nn.ReLU())
        self.regressor=nn.Sequential(nn.Linear(feat_dim+32,128),nn.ReLU(),nn.Linear(128,1))
    def forward(self,f,meta):
        a,_=self.attn(f,f,f); subj=a.mean(1)
        m=self.meta_fc(meta)
        out=self.regressor(torch.cat([subj,m],1)).squeeze(-1)
        return out

def forward_feats(backbone,X):
    B,K=X.shape[:2]
    X=X.view(B*K,N_CHANS,CROP_T)
    f=backbone(X).view(B*K,-1) ###
    return f.view(B,K,-1)

# ============================================================
# 5. LOSO CROSS-VALIDATION (subject-wise)
# ============================================================

subjects = np.array(sorted(train_subj_to_idx.keys()))
n_subjects = len(subjects)
print(f"Running LOSO CV across {n_subjects} subjects...")

y_mean, y_std = labels_df["p_factor"].mean(), labels_df["p_factor"].std() + 1e-6
def norm_y(y): return (y - y_mean) / y_std
def denorm_y(y): return y * y_std + y_mean

import warnings
warnings.filterwarnings("ignore", category=RuntimeWarning)

from sklearn.metrics import r2_score

def safe_r2_score(y_true, y_pred):
    """R² 계산 시 분산 0으로 인한 NaN 방지."""
    y_true = np.asarray(y_true)
    y_pred = np.asarray(y_pred)
    if y_true.size < 2 or np.var(y_true) < 1e-8:
        return 0.0  # 분산이 거의 없으면 R²=0 처리
    return r2_score(y_true, y_pred)


def evaluate(backbone, head, loader):
    backbone.eval(); head.eval()
    preds, trues = [], []
    with torch.no_grad():
        for X, y, meta, subj in loader:
            X, meta = X.to(DEVICE), meta.to(DEVICE)
            f = forward_feats(backbone, X)
            p = head(f, meta)
            preds.extend(denorm_y(p.cpu().numpy()))
            trues.extend(y.numpy())
    preds, trues = np.array(preds), np.array(trues)
    return (
        mean_absolute_error(trues, preds),
        mean_squared_error(trues, preds),
        safe_r2_score(trues, preds)
    )


# 각 subject를 validation으로 두고 학습 반복
loso_results = []

for fold, val_subj in enumerate(subjects, 1):
    print(f"\n========== LOSO Fold {fold}/{n_subjects} | val_subj: {val_subj} ==========")

    # subject 분할
    train_subjs = [s for s in subjects if s != val_subj]

    # 해당 subject에 해당하는 index 추출
    tr_index = [t for t in train_index if t[2] in train_subjs]
    va_index = [t for t in train_index if t[2] == val_subj]

    if len(va_index) == 0:
        print(f"⚠️ Skip {val_subj}: no training windows.")
        continue

    tr_base = WindowDataset(CACHE_DIR, tr_index, labels_df["p_factor"], CROP_T, train=True)
    va_base = WindowDataset(CACHE_DIR, va_index, labels_df["p_factor"], CROP_T, train=False)

    tr_ds = SubjectDataset(tr_base, subj_to_indices(tr_index), meta_extra, K_WINDOWS, train=True)
    va_ds = SubjectDataset(va_base, subj_to_indices(va_index), meta_extra, K_WINDOWS, train=False)

    tr_loader = DataLoader(tr_ds, batch_size=BATCH_TRAIN, shuffle=True, num_workers=4)
    va_loader = DataLoader(va_ds, batch_size=1, shuffle=False, num_workers=2)

    # 모델 초기화
    backbone = EEGConformer(n_chans=N_CHANS, n_outputs=1, n_times=CROP_T, sfreq=SFREQ).to(DEVICE)
    name, par = find_last_linear_and_parent(backbone); setattr(par, name, nn.Identity())
    feat_dim = get_feat_dim(backbone)
    head = SubjectHead(feat_dim, 3, 4).to(DEVICE)
    opt = torch.optim.AdamW(list(backbone.parameters()) + list(head.parameters()), lr=LR, weight_decay=WD)

    best_mae, best_mse, best_r2 = np.inf, np.inf, -np.inf
    for ep in range(1, EPOCHS + 1):
        backbone.train(); head.train()
        train_mae, train_mse, n_batches = 0.0, 0.0, 0

        for X, y, meta, subj in tqdm(tr_loader, desc=f"Train Fold{fold} Ep{ep}"):
            X, meta = X.to(DEVICE), meta.to(DEVICE)
            y_n = torch.tensor(norm_y(y.numpy()), dtype=torch.float32, device=DEVICE)

            opt.zero_grad()
            f = forward_feats(backbone, X)
            p = head(f, meta)
            loss_mae = F.l1_loss(p, y_n)
            loss_mse = F.mse_loss(p, y_n)
            loss = 0.5 * (loss_mae + loss_mse)
            loss.backward(); opt.step()

            train_mae += loss_mae.item()
            train_mse += loss_mse.item()
            n_batches += 1

        train_mae /= n_batches
        train_mse /= n_batches

        val_mae, val_mse, _ = evaluate(backbone, head, va_loader)
        print(f"[Fold {fold}] Ep{ep:02d} | "
              f"Train MAE={train_mae:.4f}, MSE={train_mse:.4f} | "
              f"Val MAE={val_mae:.4f}, MSE={val_mse:.4f}")

        if val_mae < best_mae:
            best_mae, best_mse, best_r2 = val_mae, val_mse, val_r2
            torch.save({
                "backbone": backbone.state_dict(),
                "head": head.state_dict(),
                "val_subj": val_subj
            }, f"loso_best_{val_subj}.pth")

    loso_results.append([val_subj, best_mae, best_mse, best_r2])
    print(f"[Fold {fold}] {val_subj} BEST -> MAE={best_mae:.4f}, MSE={best_mse:.4f}")

# 모든 fold 결과 요약
loso_df = pd.DataFrame(loso_results, columns=["subject", "MAE", "MSE", "R2"])
print("\n===== LOSO Summary =====")
print(loso_df)
print("\nAverage:", loso_df[["MAE","MSE","R2"]].mean())


# ============================================================
# 6. FINAL RELEASE-5 VALIDATION (no CV, single evaluation)
# ============================================================
print("\n=== FINAL release-5 VAL SET EVALUATION ===")

# release-5 dataset
val_base = WindowDataset(CACHE_DIR, val_index, labels_df["p_factor"], CROP_T, train=False)
val_ds   = SubjectDataset(val_base, val_subj_to_idx, meta_extra, K_WINDOWS, train=False)
val_loader = DataLoader(val_ds, batch_size=BATCH_TRAIN, shuffle=False, num_workers=2)

MODEL_DIR = '/home/RA/EEG_Challenge/Challenge2/loso_path/'

# 모든 LOSO 모델 불러와 평균 예측
all_preds, all_trues = [], []
for ckpt_name in sorted([f for f in os.listdir(MODEL_DIR) if f.startswith("loso_best_")]):
    ckpt_path = os.path.join(MODEL_DIR, ckpt_name)  # ✅ 절대경로 생성
    ckpt = torch.load(ckpt_path, map_location=DEVICE)
    backbone = EEGConformer(n_chans=N_CHANS, n_outputs=1, n_times=CROP_T, sfreq=SFREQ).to(DEVICE)
    name, par = find_last_linear_and_parent(backbone)
    setattr(par, name, nn.Identity())
    feat_dim = get_feat_dim(backbone)
    head = SubjectHead(feat_dim, 3, 4).to(DEVICE)
    backbone.load_state_dict(ckpt["backbone"], strict=False)
    head.load_state_dict(ckpt["head"], strict=False)

    # 각 모델 예측 누적
    preds_tmp, trues_tmp = [], []
    with torch.no_grad():
        for X, y, meta, subj in val_loader:
            X, meta = X.to(DEVICE), meta.to(DEVICE)
            f = forward_feats(backbone, X)
            p = head(f, meta)
            preds_tmp.extend(p.cpu().numpy())
            trues_tmp.extend(y.numpy())
    all_preds.append(preds_tmp)
    all_trues = trues_tmp  # 동일하므로 한 번만 저장

# 모델별 예측 평균
all_preds = np.mean(np.stack(all_preds, axis=0), axis=0)
all_trues = np.array(all_trues)

# 최종 release-5 성능
mae = mean_absolute_error(all_trues, denorm_y(all_preds))
mse = mean_squared_error(all_trues, denorm_y(all_preds))
r2  = safe_r2_score(all_trues, denorm_y(all_preds))
print(f"\nFinal release-5 performance | MAE={mae:.4f} | MSE={mse:.4f} | R²={r2:.3f}")



Device: cuda
Subjects with p_factor: 3116
Train cache: 309, Val cache: 8


Indexing windows: 100%|██████████| 8/8 [00:00<00:00, 180.69it/s]


Train subjects: 298, Val subjects: 8
Running LOSO CV across 298 subjects...

========== LOSO Fold 1/298 | val_subj: sub-NDARAA306NT2 ==========


Train Fold1 Ep1: 100%|██████████| 38/38 [00:03<00:00, 10.03it/s]


[Fold 1] Ep01 | Train MAE=0.7954, MSE=0.9953 | Val MAE=0.9011, MSE=0.8120


...

[Fold 298] Ep08 | Train MAE=0.8034, MSE=1.0235 | Val MAE=1.3627, MSE=1.8570


Train Fold298 Ep9: 100%|██████████| 38/38 [00:02<00:00, 14.31it/s]


[Fold 298] Ep09 | Train MAE=0.7773, MSE=0.9727 | Val MAE=1.3517, MSE=1.8271


Train Fold298 Ep10: 100%|██████████| 38/38 [00:02<00:00, 14.16it/s]


[Fold 298] Ep10 | Train MAE=0.7889, MSE=0.9916 | Val MAE=1.3755, MSE=1.8920
[Fold 298] sub-NDARZZ404ZPN BEST -> MAE=1.3517, MSE=1.8271

===== LOSO Summary =====
              subject       MAE       MSE  R2
0    sub-NDARAA306NT2  0.806474  0.650400 NaN
1    sub-NDARAA504CRN  0.402680  0.162151 NaN
2    sub-NDARAH129BMC  1.839450  3.383574 NaN
3    sub-NDARAK770XEW  0.219453  0.048160 NaN
4    sub-NDARAK772VFJ  0.540246  0.291865 NaN
..                ...       ...       ...  ..
293  sub-NDARZT957CWG  1.634668  2.672140 NaN
294  sub-NDARZV421TCZ  0.250428  0.062714 NaN
295  sub-NDARZW426CT2  1.062215  1.128301 NaN
296  sub-NDARZX827NWT  1.042550  1.086911 NaN
297  sub-NDARZZ404ZPN  1.351695  1.827078 NaN

[298 rows x 4 columns]

Average: MAE    0.649039
MSE    0.719006
R2          NaN
dtype: float64

=== FINAL release-5 VAL SET EVALUATION ===

Final release-5 performance | MAE=0.8727 | MSE=1.1800 | R²=0.024


In [57]:
# ============================================================
# FINAL TRAINING & RELEASE-5 VALIDATION (no CV)
# ============================================================

MODEL_DIR = '/home/RA/EEG_Challenge/Challenge2/final_model'
os.makedirs(MODEL_DIR, exist_ok=True)

print("\n=== FINAL TRAINING ON ALL TRAIN SUBJECTS ===")

# 1️⃣ 모든 train subject 사용
train_all_index = train_index  # 모든 train subjects 포함
val_index_final = val_index    # release-5 set

# Dataset 구성
tr_base = WindowDataset(CACHE_DIR, train_all_index, labels_df["p_factor"], CROP_T, train=True)
va_base = WindowDataset(CACHE_DIR, val_index_final, labels_df["p_factor"], CROP_T, train=False)

tr_ds = SubjectDataset(tr_base, subj_to_indices(train_all_index), meta_extra, K_WINDOWS, train=True)
va_ds = SubjectDataset(va_base, subj_to_indices(val_index_final), meta_extra, K_WINDOWS, train=False)

tr_loader = DataLoader(tr_ds, batch_size=BATCH_TRAIN, shuffle=True, num_workers=4)
va_loader = DataLoader(va_ds, batch_size=BATCH_TRAIN, shuffle=False, num_workers=2)

# 2️⃣ 모델 초기화
backbone = EEGConformer(n_chans=N_CHANS, n_outputs=1, n_times=CROP_T, sfreq=SFREQ).to(DEVICE)
name, par = find_last_linear_and_parent(backbone); setattr(par, name, nn.Identity())
feat_dim = get_feat_dim(backbone)
head = SubjectHead(feat_dim, 3, 4).to(DEVICE)
opt = torch.optim.AdamW(list(backbone.parameters()) + list(head.parameters()), lr=LR, weight_decay=WD)

best_mae, best_mse, best_r2 = np.inf, np.inf, -np.inf
model_path = os.path.join(MODEL_DIR, "final_model_all_train.pth")

# 3️⃣ 학습 루프
for ep in range(1, EPOCHS + 1):
    backbone.train(); head.train()
    train_mae, train_mse, n_batches = 0.0, 0.0, 0

    for X, y, meta, subj in tqdm(tr_loader, desc=f"[FINAL TRAIN] Epoch {ep}/{EPOCHS}"):
        X, meta = X.to(DEVICE), meta.to(DEVICE)
        y_n = torch.tensor(norm_y(y.numpy()), dtype=torch.float32, device=DEVICE)

        opt.zero_grad()
        f = forward_feats(backbone, X)
        p = head(f, meta)
        loss_mae = F.l1_loss(p, y_n)
        loss_mse = F.mse_loss(p, y_n)
        loss = 0.5 * (loss_mae + loss_mse)
        loss.backward(); opt.step()

        train_mae += loss_mae.item()
        train_mse += loss_mse.item()
        n_batches += 1

    train_mae /= n_batches
    train_mse /= n_batches

    # release-5 검증
    val_mae, val_mse, val_r2 = evaluate(backbone, head, va_loader)
    print(f"[Ep {ep:02d}] Train MAE={train_mae:.4f}, MSE={train_mse:.4f} | "
          f"Val MAE={val_mae:.4f}, MSE={val_mse:.4f}, R²={val_r2:.3f}")

    if val_mae < best_mae:
        best_mae, best_mse, best_r2 = val_mae, val_mse, val_r2
        torch.save({
            "backbone": backbone.state_dict(),
            "head": head.state_dict(),
            "epoch": ep,
            "metrics": {"MAE": best_mae, "MSE": best_mse, "R2": best_r2}
        }, model_path)
        print(f"✅ Saved best model at epoch {ep} -> {model_path}")

# 4️⃣ release-5 최종 평가
print("\n=== FINAL RELEASE-5 EVALUATION ===")
ckpt = torch.load(model_path, map_location=DEVICE)
backbone = EEGConformer(n_chans=N_CHANS, n_outputs=1, n_times=CROP_T, sfreq=SFREQ).to(DEVICE)
name, par = find_last_linear_and_parent(backbone); setattr(par, name, nn.Identity())
feat_dim = get_feat_dim(backbone)
head = SubjectHead(feat_dim, 3, 4).to(DEVICE)
backbone.load_state_dict(ckpt["backbone"], strict=False)
head.load_state_dict(ckpt["head"], strict=False)

mae, mse, r2 = evaluate(backbone, head, va_loader)
print(f"Final release-5 performance | MAE={mae:.4f} | MSE={mse:.4f} | R²={r2:.3f}")

# 5️⃣ 최종 모델 경로 저장
import json, datetime
stamp = datetime.datetime.now().strftime("%Y%m%d_%H%M%S")
log_file = os.path.join(MODEL_DIR, f"final_model_path_{stamp}.json")
with open(log_file, "w") as f:
    json.dump({
        "model_path": model_path,
        "timestamp": stamp,
        "final_scores": {"MAE": mae, "MSE": mse, "R2": float(r2)}
    }, f, indent=2)

print(f"\n✅ Final model path saved at: {log_file}")



=== FINAL TRAINING ON ALL TRAIN SUBJECTS ===


[FINAL TRAIN] Epoch 1/10: 100%|██████████| 38/38 [00:03<00:00,  9.53it/s]


[Ep 01] Train MAE=0.7841, MSE=0.9900 | Val MAE=0.8601, MSE=1.1694, R²=0.033
✅ Saved best model at epoch 1 -> /home/RA/EEG_Challenge/Challenge2/final_model/final_model_all_train.pth


[FINAL TRAIN] Epoch 2/10: 100%|██████████| 38/38 [00:03<00:00,  9.86it/s]


[Ep 02] Train MAE=0.7829, MSE=0.9877 | Val MAE=0.8662, MSE=1.1647, R²=0.037


...

[Ep 08] Train MAE=0.7883, MSE=0.9855 | Val MAE=0.8559, MSE=1.1513, R²=0.048


[FINAL TRAIN] Epoch 9/10: 100%|██████████| 38/38 [00:06<00:00,  5.80it/s]


[Ep 09] Train MAE=0.7758, MSE=0.9778 | Val MAE=0.8644, MSE=1.1576, R²=0.043


[FINAL TRAIN] Epoch 10/10: 100%|██████████| 38/38 [00:06<00:00,  5.52it/s]


[Ep 10] Train MAE=0.7857, MSE=0.9799 | Val MAE=0.8648, MSE=1.1660, R²=0.036

=== FINAL RELEASE-5 EVALUATION ===
Final release-5 performance | MAE=0.8569 | MSE=1.1703 | R²=0.032

✅ Final model path saved at: /home/RA/EEG_Challenge/Challenge2/final_model/final_model_path_20251021_175732.json


## ContraWR

In [78]:
# ============================================================
# CHALLENGE 2: p-factor regression (ContraWR backbone)
# 128채널 + Cz 제거 + Subject-level meta conditioning
# Single train/val (release-5 validation)
# ============================================================

import os, gc, random
import numpy as np
import pandas as pd
from collections import defaultdict

import torch
import torch.nn as nn
import torch.nn.functional as F
from torch.utils.data import Dataset, DataLoader
from tqdm import tqdm
from sklearn.metrics import mean_absolute_error, mean_squared_error, r2_score

from braindecode.models import ContraWR

# -----------------------------
# CONFIGURATION
# -----------------------------
CACHE_DIR = "/data5/open_data/HBN/cache_eeg_windows"
BIDS_ROOT = "/data5/open_data/HBN/EEG_BIDS"
DATASETS  = [f"ds0055{i:02d}" for i in range(5, 17)]

DEVICE   = "cuda" if torch.cuda.is_available() else "cpu"
SFREQ    = 125
N_CHANS  = 128    # ✅ 128채널 (Cz 제거)
CROP_T   = 800
K_WINDOWS = 4

BATCH_TRAIN = 8
EPOCHS = 15
LR = 2e-4
WD = 1e-3
SEED = 42

# -----------------------------
# SET SEED
# -----------------------------
def set_seed(seed=SEED):
    random.seed(seed); np.random.seed(seed)
    torch.manual_seed(seed); torch.cuda.manual_seed_all(seed)
    torch.backends.cudnn.benchmark = True
set_seed()
print(f"Device: {DEVICE}")

# ============================================================
# 1. PARTICIPANTS & LABELS
# ============================================================
def load_all_participants(bids_root=BIDS_ROOT, datasets=DATASETS):
    dfs = []
    for ds in datasets:
        pfile = os.path.join(bids_root, ds, "participants.tsv")
        if os.path.exists(pfile):
            df = pd.read_csv(pfile, sep="\t")
            df["release"] = ds
            dfs.append(df)
    df_all = pd.concat(dfs, ignore_index=True)
    df_all.columns = [c.lower() for c in df_all.columns]
    df_all = df_all.drop_duplicates(subset=["participant_id"]).set_index("participant_id")
    if "p_factor" not in df_all.columns:
        alt = [c for c in df_all.columns if "p" in c and "factor" in c]
        if alt: df_all = df_all.rename(columns={alt[0]:"p_factor"})
    for col in ["age","sex","ehq_total"]:
        if col not in df_all.columns:
            df_all[col] = np.nan
    return df_all

meta_df = load_all_participants()
labels_df = meta_df[["p_factor"]].dropna()
meta_extra = meta_df[["age","sex","ehq_total"]].copy()

# sex → numeric, 나머지는 표준화
meta_extra["sex"] = meta_extra["sex"].map({"M":1,"F":0,"m":1,"f":0}).fillna(0.5)
for col in ["age","ehq_total"]:
    meta_extra[col] = meta_extra[col].fillna(meta_extra[col].mean())
    m,s = meta_extra[col].mean(), meta_extra[col].std()+1e-6
    meta_extra[col] = (meta_extra[col]-m)/s

print(f"Subjects with p_factor: {len(labels_df)}")

# ============================================================
# 2. CACHED FILES
# ============================================================
all_cache = [f for f in os.listdir(CACHE_DIR) if f.endswith(".npy")]
train_files = [f for f in all_cache if "_train_" in f]
val_files   = [f for f in all_cache if "_val_" in f]   # release-5
print(f"Train cache: {len(train_files)}, Val cache: {len(val_files)}")

def subj_from_fname(fname): return fname.split("_")[0]

def build_window_index(cache_dir, cache_files, label_index):
    idx=[]
    for fname in tqdm(cache_files, desc="Indexing windows"):
        subj=subj_from_fname(fname)
        if subj not in label_index: continue
        arr=np.load(os.path.join(cache_dir,fname),mmap_mode="r")
        for s in range(arr.shape[0]): idx.append((fname,s,subj))
        del arr
    return idx

train_index = build_window_index(CACHE_DIR, train_files, labels_df.index)
val_index   = build_window_index(CACHE_DIR, val_files,   labels_df.index)

def subj_to_indices(index_list):
    d=defaultdict(list)
    for i,(_,_,subj) in enumerate(index_list): d[subj].append(i)
    return d

train_subj_to_idx=subj_to_indices(train_index)
val_subj_to_idx=subj_to_indices(val_index)
print(f"Train subjects: {len(train_subj_to_idx)}, Val subjects: {len(val_subj_to_idx)}")

# ============================================================
# 3. DATASET DEFINITIONS
# ============================================================
class WindowDataset(Dataset):
    """129채널 캐시에서 Cz 제거 → 128채널 EEG 반환"""
    def __init__(self, cache_dir, index, labels, crop_len=800, train=True):
        self.cache_dir = cache_dir
        self.index = index
        self.labels = labels
        self.crop_len = crop_len
        self.train = train

    def __getitem__(self, i):
        fname, sidx, subj = self.index[i]
        y = float(self.labels.loc[subj])

        x = np.load(os.path.join(self.cache_dir, fname), mmap_mode="r")[sidx]  # (129, T)
        if x.shape[0] == 129:
            x = x[:-1, :]   # ✅ Cz 제거 → (128, T)

        if x.shape[1] > self.crop_len:
            start = np.random.randint(0, x.shape[1] - self.crop_len + 1) if self.train else (x.shape[1]-self.crop_len)//2
            x = x[:, start:start+self.crop_len]

        # normalization
        x = (x - x.mean(axis=1, keepdims=True)) / (x.std(axis=1, keepdims=True) + 1e-6)
        return torch.tensor(x).float(), torch.tensor(y), subj

class SubjectDataset(Dataset):
    """subject별로 K개 window anchor + meta conditioning"""
    def __init__(self, base_ds, subj_to_idx, meta, K=4, train=True):
        self.base_ds=base_ds; self.subj_to_idx=subj_to_idx
        self.subjs=list(subj_to_idx.keys()); self.meta=meta
        self.K=K; self.train=train
    def __len__(self): return len(self.subjs)
    def __getitem__(self, i):
        subj=self.subjs[i]; idxs=self.subj_to_idx[subj]
        sel=random.choices(idxs,k=self.K)
        Xs, ys = [], []
        for j in sel:
            x,y,s=self.base_ds[j]; Xs.append(x); ys.append(y)
        X=torch.stack(Xs,dim=0) # (K,C,T)
        meta_vec=torch.tensor(self.meta.loc[subj].values,dtype=torch.float32)
        return X, ys[0], meta_vec, subj

# ============================================================
# 4. MODEL COMPONENTS
# ============================================================
def find_last_linear_and_parent(model):
    """마지막 Linear layer 제거용"""
    target=None; parent=None
    for n,m in model.named_modules():
        if isinstance(m,nn.Linear): target=n
    if target:
        parent=model
        parts=target.split(".")
        for p in parts[:-1]: parent=getattr(parent,p)
        return parts[-1],parent
    return None,None

def get_feat_dim(model):
    """dummy input으로 feature dim 계산"""
    x=torch.zeros((2, N_CHANS, CROP_T), device=DEVICE)
    with torch.no_grad():
        y=model(x)
        y=y.view(y.size(0),-1)
    return y.size(1)

class SubjectHead(nn.Module):
    def __init__(self, feat_dim, meta_dim=3, heads=4):
        super().__init__()
        # feat_dim이 heads로 나눠떨어지지 않으면 자동조정
        if feat_dim % heads != 0:
            heads = 1
        self.attn = nn.MultiheadAttention(feat_dim, heads, batch_first=True)
        self.meta_fc = nn.Sequential(nn.Linear(meta_dim, 32), nn.ReLU())
        self.regressor = nn.Sequential(nn.Linear(feat_dim+32, 128), nn.ReLU(), nn.Linear(128, 1))
    def forward(self, f, meta):
        a,_ = self.attn(f,f,f)
        subj = a.mean(1)
        m = self.meta_fc(meta)
        out = self.regressor(torch.cat([subj,m],1)).squeeze(-1)
        return out

def forward_feats(backbone, X):
    """(B,K,C,T) → (B,K,F)"""
    if X.ndim == 4:
        B,K,C,T = X.shape
        X = X.view(B*K, C, T)
        f = backbone(X)
        f = f.view(B,K,-1)
        return f
    elif X.ndim == 3:
        B,C,T = X.shape
        f = backbone(X)
        return f.unsqueeze(1)
    else:
        raise ValueError(f"Unexpected input shape: {X.shape}")

def evaluate(backbone, head, loader):
    backbone.eval(); head.eval()
    preds,trues=[],[]
    with torch.no_grad():
        for X,y,meta,subj in loader:
            X,meta = X.to(DEVICE),meta.to(DEVICE)
            f = forward_feats(backbone,X)
            p = head(f,meta)
            preds.extend(p.cpu().numpy()); trues.extend(y.numpy())
    return (
        mean_absolute_error(trues,preds),
        mean_squared_error(trues,preds),
        r2_score(trues,preds)
    )

# ============================================================
# 5. TRAINING
# ============================================================
train_base = WindowDataset(CACHE_DIR, train_index, labels_df["p_factor"], CROP_T, train=True)
val_base   = WindowDataset(CACHE_DIR, val_index,   labels_df["p_factor"], CROP_T, train=False)
train_ds   = SubjectDataset(train_base, train_subj_to_idx, meta_extra, K_WINDOWS, train=True)
val_ds     = SubjectDataset(val_base,   val_subj_to_idx,   meta_extra, K_WINDOWS, train=False)

train_loader = DataLoader(train_ds, batch_size=BATCH_TRAIN, shuffle=True, num_workers=4)
val_loader   = DataLoader(val_ds,   batch_size=BATCH_TRAIN, shuffle=False, num_workers=2)

# ------------------------------------------------------------
# Model Init
# ------------------------------------------------------------
backbone = ContraWR(
    n_chans=N_CHANS,
    n_outputs=1,   # ✅ feature extractor 모드
    n_times=CROP_T,
    sfreq=SFREQ
).to(DEVICE)

# 마지막 회귀층 제거
name, par = find_last_linear_and_parent(backbone)
if name is not None:
    setattr(par, name, nn.Identity())

feat_dim = get_feat_dim(backbone)
print(f"Detected feature dim = {feat_dim}")

head = SubjectHead(feat_dim, meta_dim=3, heads=4).to(DEVICE)
opt = torch.optim.AdamW(list(backbone.parameters()) + list(head.parameters()), lr=LR, weight_decay=WD)

best_mae, best_mse, best_r2 = np.inf, np.inf, -np.inf

for ep in range(1, EPOCHS+1):
    backbone.train(); head.train()
    train_mae, train_mse, n_batches = 0.0, 0.0, 0

    for X,y,meta,subj in tqdm(train_loader, desc=f"Train Ep{ep}"):
        X,meta = X.to(DEVICE),meta.to(DEVICE)
        y = y.to(torch.float32).to(DEVICE)
        opt.zero_grad()
        f = forward_feats(backbone,X)
        p = head(f,meta)
        loss_mae = F.l1_loss(p,y)
        loss_mse = F.mse_loss(p,y)
        loss = 0.5*(loss_mae+loss_mse)
        loss.backward(); opt.step()
        train_mae += loss_mae.item(); train_mse += loss_mse.item(); n_batches += 1

    train_mae /= n_batches; train_mse /= n_batches
    val_mae,val_mse,val_r2 = evaluate(backbone,head,val_loader)

    print(f"[Ep{ep:02d}] Train MAE={train_mae:.4f}, MSE={train_mse:.4f} | "
          f"Val MAE={val_mae:.4f}, MSE={val_mse:.4f}, R²={val_r2:.3f}")

    if val_mae < best_mae:
        best_mae,best_mse,best_r2 = val_mae,val_mse,val_r2
        torch.save({"backbone":backbone.state_dict(),"head":head.state_dict()},
                   "best_contrawr_128ch.pth")

print(f"\n✅ Best Validation Performance → MAE={best_mae:.4f}, MSE={best_mse:.4f}, R²={best_r2:.3f}")


Device: cuda
Subjects with p_factor: 3116
Train cache: 310, Val cache: 8


Indexing windows: 100%|██████████| 8/8 [00:00<00:00, 19.09it/s]


Train subjects: 299, Val subjects: 8
Detected feature dim = 256


Train Ep1: 100%|██████████| 38/38 [00:06<00:00,  6.13it/s]


[Ep01] Train MAE=0.7312, MSE=0.8490 | Val MAE=0.8659, MSE=1.3660, R²=-0.130


...

[Ep13] Train MAE=0.7163, MSE=0.8135 | Val MAE=0.8686, MSE=1.2949, R²=-0.071


Train Ep14: 100%|██████████| 38/38 [00:06<00:00,  6.11it/s]


[Ep14] Train MAE=0.7283, MSE=0.8274 | Val MAE=0.8993, MSE=1.3800, R²=-0.141


Train Ep15: 100%|██████████| 38/38 [00:04<00:00,  9.04it/s]


[Ep15] Train MAE=0.7216, MSE=0.8176 | Val MAE=0.8629, MSE=1.2545, R²=-0.038

✅ Best Validation Performance → MAE=0.8483, MSE=1.1734, R²=0.030


## CbraMod

In [2]:
import os, random, time, warnings, json
import numpy as np, pandas as pd, torch, mne
import torch.nn as nn
import torch.nn.functional as F
from torch.utils.data import Dataset, DataLoader
from tqdm import tqdm

# ============================================================
# CONFIG
# ============================================================
BIDS_ROOT   = "/data5/open_data/HBN/EEG_BIDS"
CACHE_DIR = "/data5/open_data/HBN/cache_eeg_windows"
MISSING_TXT = "/home/RA/EEG_Challenge/Challenge2/logs/missing_cache_files.txt"
CBRAMOD_WEIGHTS = "/home/RA/EEG_Challenge/Challenge2/CBraMod/pretrained_weights/pretrained_weights.pth"

DEVICE = "cuda" if torch.cuda.is_available() else "cpu"
SFREQ  = 100
CROP_T = 1000
N_CHANS = 128   # Cz 제거 후
BATCH   = 8
EPOCHS  = 10
LR_FRONT, LR_BACKB, LR_HEAD = 3e-4, 1e-4, 3e-4
WD = 1e-2
SEED = 42
VAL_RELEASE = "R5"

warnings.filterwarnings("ignore")
mne.set_log_level("CRITICAL")
random.seed(SEED); np.random.seed(SEED); torch.manual_seed(SEED); torch.cuda.manual_seed_all(SEED)

# ============================================================
# 1. 파일 리스트 구성
# ============================================================
def build_eeg_filelist(cache_dir, missing_txt, bids_root):
    cache_files = [f for f in os.listdir(cache_dir) if f.endswith(".npy")]
    found_from_cache, not_found = [], []

    # -----------------------------
    # ✅ 1) CACHE FILES 처리
    # -----------------------------
    for f in cache_files:
        subj = f.split("_")[0]  # sub-XXXX
        task = f.split("_task-")[1].split("_")[0]
        run = None
        if "_run-" in f:
            run = f.split("_run-")[1].split("_")[0]
        matched = False

        for ds in os.listdir(bids_root):
            eeg_dir = os.path.join(bids_root, ds, subj, "eeg")
            if not os.path.exists(eeg_dir):
                continue

            for fname in os.listdir(eeg_dir):
                if not fname.endswith(".set"):
                    continue
                # run 없는 경우
                if run is None:
                    if f"task-{task}" in fname:
                        found_from_cache.append(os.path.join(eeg_dir, fname))
                        matched = True
                        break
                else:
                    # run 있는 경우: 1 / 01 패딩 모두 허용
                    run_variants = [f"run-{run}", f"run-0{run}"]
                    if f"task-{task}" in fname and any(r in fname for r in run_variants):
                        found_from_cache.append(os.path.join(eeg_dir, fname))
                        matched = True
                        break
            if matched:
                break
        if not matched:
            not_found.append(f)

    # -----------------------------
    # ✅ 2) MISSING FILES 처리
    # -----------------------------
    found_from_missing = []
    if os.path.exists(missing_txt):
        with open(missing_txt, "r") as f:
            for line in f:
                base = line.strip()
                if len(base) == 0:
                    continue
                subj = base.split("_task-")[0]
                task = base.split("_task-")[1].split("_")[0]
                run = None
                if "_run-" in base:
                    run = base.split("_run-")[1].split("_")[0]

                for ds in os.listdir(bids_root):
                    eeg_dir = os.path.join(bids_root, ds, subj, "eeg")
                    if not os.path.exists(eeg_dir):
                        continue

                    for fname in os.listdir(eeg_dir):
                        if not fname.endswith(".set"):
                            continue
                        if run is None:
                            if f"task-{task}" in fname:
                                found_from_missing.append(os.path.join(eeg_dir, fname))
                                break
                        else:
                            run_variants = [f"run-{run}", f"run-0{run}"]
                            if f"task-{task}" in fname and any(r in fname for r in run_variants):
                                found_from_missing.append(os.path.join(eeg_dir, fname))
                                break

    # -----------------------------
    # ✅ 3) 최종 결과
    # -----------------------------
    all_files = sorted(list(set(found_from_cache + found_from_missing)))
    print(f"✅ Total EEG files: {len(all_files)} "
          f"(cache={len(found_from_cache)}, missing={len(found_from_missing)})")
    if len(not_found) > 0:
        print(f"⚠️ {len(not_found)} cache-based files not matched to .set:")
        print("\n".join(not_found[:15]), "...")
    return all_files


# ============================================================
# 2. 메타데이터 로드
# ============================================================
def load_participants(bids_root):
    dfs=[]
    for ds in sorted(os.listdir(bids_root)):
        pfile=os.path.join(bids_root, ds, "participants.tsv")
        if os.path.exists(pfile):
            df=pd.read_csv(pfile,sep="\t")
            df["release_number"]=ds
            dfs.append(df)
    meta=pd.concat(dfs,ignore_index=True)
    meta.columns=[c.lower() for c in meta.columns]
    meta=meta.drop_duplicates(subset=["participant_id"]).set_index("participant_id")

    # p_factor, age, sex, ehq_total 정리
    if "p_factor" not in meta.columns:
        alt=[c for c in meta.columns if "p" in c and "factor" in c]
        if alt: meta=meta.rename(columns={alt[0]:"p_factor"})
    meta["sex"]=meta["sex"].map({"M":1,"F":0,"m":1,"f":0}).fillna(0.5)
    for c in ["age","ehq_total"]:
        if c not in meta.columns: meta[c]=np.nan
        meta[c]=meta[c].fillna(meta[c].mean())
        m,s=meta[c].mean(),meta[c].std()+1e-8
        meta[c]=(meta[c]-m)/s
    return meta

# ============================================================
# 3. Dataset
# ============================================================
def subj_from_fname(fname):
    base=os.path.basename(fname)
    return base.split("_")[0]

class RawEEGDataset(Dataset):
    def __init__(self, files, meta, train=True):
        self.files=files; self.meta=meta; self.train=train
        self.targets=meta["p_factor"].dropna()
        self.subj_ids=set(self.targets.index)

    def __len__(self): return len(self.files)

    def __getitem__(self,i):
        path=self.files[i]
        subj=subj_from_fname(path)
        if subj not in self.subj_ids:
            return torch.zeros((N_CHANS,CROP_T)), torch.tensor(0.), torch.zeros(3)
        raw=mne.io.read_raw_eeglab(path,preload=True,verbose=False)
        raw.filter(0.5,50.,fir_design="firwin")
        raw.set_eeg_reference("average",projection=False)
        raw.resample(SFREQ)
        x=raw.get_data()
        if x.shape[0]==129: x=x[:-1,:]  # Cz 제거
        x=(x-x.mean(axis=1,keepdims=True))/(x.std(axis=1,keepdims=True)+1e-6)

        T=x.shape[1]
        if T<CROP_T:
            pad=np.zeros((x.shape[0],CROP_T)); pad[:,:T]=x; x=pad
        else:
            st=np.random.randint(0,T-CROP_T+1) if self.train else (T-CROP_T)//2
            x=x[:,st:st+CROP_T]

        y=float(self.meta.loc[subj,"p_factor"]) if subj in self.meta.index else 0.
        meta_vec=torch.tensor([
            self.meta.loc[subj,"age"],
            self.meta.loc[subj,"sex"],
            self.meta.loc[subj,"ehq_total"]
        ],dtype=torch.float32) if subj in self.meta.index else torch.zeros(3)
        return torch.tensor(x,dtype=torch.float32), torch.tensor(y,dtype=torch.float32), meta_vec

# ============================================================
# 4. 모델 정의 (CBraMod + regression head)
# ============================================================
class SincConv1d(nn.Module):
    def __init__(self, out_channels=64, kernel_size=129, sample_rate=100, min_hz=0.3, max_hz=45.0):
        super().__init__()
        self.out_channels = out_channels
        self.kernel_size  = kernel_size
        self.sample_rate  = sample_rate
        self.min_hz       = float(min_hz)
        self.max_hz       = float(max_hz)
        low  = torch.linspace(self.min_hz, self.max_hz - 5.0, out_channels)
        band = torch.ones(out_channels) * 5.0
        self.low_hz_  = nn.Parameter(low)
        self.band_hz_ = nn.Parameter(band)
        n = torch.arange(-(kernel_size // 2), kernel_size // 2 + 1).float()
        self.register_buffer("n", n)

    def forward(self, x):  # x: (B, C, T)
        B, C, T = x.shape
        device  = x.device
        dtype   = x.dtype

        # low: (out,)
        low  = torch.clamp(torch.abs(self.low_hz_), min=self.min_hz, max=self.max_hz - 1.0)
        # high: clamp with tensor min/max (same shape)
        raw_high = low + torch.abs(self.band_hz_)
        min_v    = low + 1.0
        max_v    = torch.full_like(low, self.max_hz)
        high     = torch.clamp(raw_high, min=min_v, max=max_v)

        n = self.n.to(device=device, dtype=dtype)
        window = torch.hamming_window(self.kernel_size, periodic=False, dtype=dtype, device=device)

        filters = []
        nyq = (self.sample_rate / 2.0)
        for i in range(self.out_channels):
            f1 = low[i]  / nyq
            f2 = high[i] / nyq
            h1 = 2 * f2 * torch.sinc(2 * f2 * n)
            h2 = 2 * f1 * torch.sinc(2 * f1 * n)
            bandpass = (h1 - h2) * window
            filters.append(bandpass)
        filt = torch.stack(filters, dim=0).unsqueeze(1)  # (out, 1, K)

        x_dw = x.view(B * C, 1, T)
        y = F.conv1d(x, filt, stride=1, padding=self.kernel_size // 2)

        y = y.view(B, C, self.out_channels, y.shape[-1]).sum(dim=1)  # (B, out, T)
        return y


class SEBlock(nn.Module):
    def __init__(self,c,r=8):
        super().__init__()
        self.fc1=nn.Linear(c,c//r); self.fc2=nn.Linear(c//r,c)
    def forward(self,x):
        s=x.mean(-1); e=F.relu(self.fc1(s)); e=torch.sigmoid(self.fc2(e)).unsqueeze(-1)
        return x*e

class CBraModBackbone(nn.Module):
    def __init__(self,out_dim=512):
        super().__init__()
        self.conv=nn.Sequential(
            nn.Conv1d(64,128,7,padding=3),nn.ReLU(),
            nn.Conv1d(128,256,5,padding=2),nn.ReLU(),
            nn.AdaptiveAvgPool1d(1))
        self.fc=nn.Linear(256,out_dim); self.out_dim=out_dim
    def load_weights(self,wpath):
        sd=torch.load(wpath,map_location="cpu")
        self.load_state_dict(sd,strict=False)
        print(f"[Info] Loaded pretrained CBraMod weights: {wpath}")
    def forward(self,x):
        h=self.conv(x).squeeze(-1)
        return self.fc(h)

class RegressionHead(nn.Module):
    def __init__(self,in_dim,meta_dim=3,hetero=True):
        super().__init__()
        self.hetero=hetero
        self.meta_fc=nn.Sequential(nn.Linear(meta_dim,32),nn.ReLU())
        self.fc=nn.Sequential(nn.Linear(in_dim+32,128),nn.ReLU(),nn.Linear(128,1))
        if hetero:
            self.logv=nn.Sequential(nn.Linear(in_dim+32,64),nn.ReLU(),nn.Linear(64,1))
    def forward(self,f,meta):
        
        m=self.meta_fc(meta)
        z=torch.cat([f,m],1)
        
        mu=self.fc(z).squeeze(-1)
        if self.hetero:
            lv = self.logv(z)
            
            if isinstance(lv, (tuple, list)):
                lv = lv[0]
            logv = lv.squeeze(-1)
            logv = torch.clamp(logv, min=-6.0, max=6.0)

            return mu,logv

def hetero_nll(y,mu,logv):
    inv=torch.exp(-logv)
    return 0.5*((y-mu)**2*inv+logv).mean()

# ============================================================
# 5. 학습 루프
# ============================================================
def train_epoch(model,head,loader,opt,scaler,hetero):
    model.train(); head.train(); total=0
    for x,y,meta in tqdm(loader,desc="Train",leave=False):
        x,y,meta=x.to(DEVICE),y.to(DEVICE),meta.to(DEVICE)
        opt.zero_grad(set_to_none=True)
        with torch.cuda.amp.autocast(True):
            z=model(x); 
            out=head(z,meta)
            if hetero:
                mu,logv=out; loss=hetero_nll(y,mu,logv)
            else:
                mu=out; loss=F.mse_loss(mu,y)
        scaler.scale(loss).backward(); scaler.step(opt); scaler.update()
        total+=loss.item()
    return total/len(loader)

@torch.no_grad()
def evaluate(model,head,loader,hetero):
    model.eval(); head.eval()
    preds,trues=[],[]
    for x,y,meta in tqdm(loader,desc="Val",leave=False):
        x,meta=x.to(DEVICE),meta.to(DEVICE)
        out=head(model(x),meta)
        mu=out[0] if hetero else out
        preds.extend(mu.detach().cpu().numpy()); trues.extend(y.numpy())
    preds,trues=np.array(preds),np.array(trues)
    mae=np.mean(np.abs(preds-trues))
    mse=np.mean((preds-trues)**2)
    r2=1-np.sum((preds-trues)**2)/np.sum((trues-trues.mean())**2)
    nrmse=np.sqrt(mse)/(trues.std()+1e-8)
    return mae,mse,r2,nrmse

# ============================================================
# 6. 실행
# ============================================================
def main():
    files=build_eeg_filelist(CACHE_DIR, MISSING_TXT, BIDS_ROOT)
    meta=load_participants(BIDS_ROOT)
    # release split
    train_files=[f for f in files if VAL_RELEASE not in f]
    val_files  =[f for f in files if VAL_RELEASE in f]
    train_ds=RawEEGDataset(train_files,meta,train=True)
    val_ds  =RawEEGDataset(val_files,meta,train=False)
    train_loader=DataLoader(train_ds,batch_size=BATCH,shuffle=True,num_workers=4)
    val_loader  =DataLoader(val_ds,batch_size=BATCH,shuffle=False,num_workers=2)

    # Model
    front=SincConv1d(out_channels=64,kernel_size=129,sample_rate=SFREQ)
    se=SEBlock(64); backbone=CBraModBackbone(out_dim=512)
    backbone.load_weights(CBRAMOD_WEIGHTS)
    model=nn.Sequential(front,se,backbone).to(DEVICE)
    head=RegressionHead(512,meta_dim=3,hetero=True).to(DEVICE)
    opt=torch.optim.AdamW(list(model.parameters())+list(head.parameters()),
                          lr=3e-4,weight_decay=WD)
    scaler=torch.cuda.amp.GradScaler()
    best={"NRMSE":9e9}

    for ep in range(1, EPOCHS + 1):
        t0 = time.time()
        tr = train_epoch(model, head, train_loader, opt, scaler, hetero=True)
        mae, mse, r2, nrmse = evaluate(model, head, val_loader, hetero=True)
        print(f"[Ep{ep:02d}] TrainLoss={tr:.4f} | Val MSE={mse:.4f} MAE={mae:.4f} R2={r2:.3f} time={time.time()-t0:.1f}s")

        # ✅ MSE 기준으로 저장 (NRMSE 제외)
        if mse < best["NRMSE"]:   # key 이름 그대로 두되 값은 MSE
            best = {"NRMSE": float(mse), "epoch": ep}  # float32 → float 변환
            torch.save({
                "front": front.state_dict(),
                "backbone": backbone.state_dict(),
                "head": head.state_dict()
            }, "best_cbramod_raw_finetune.pth")

            # ✅ JSON 직렬화 가능한 타입으로 기록
            with open("best_metrics.json", "w") as f:
                json.dump({
                    "MAE": float(mae),
                    "MSE": float(mse),
                    "R2": float(r2)
                }, f, indent=2)

    print("✅ Best:", best)


if __name__=="__main__":
    main()


✅ Total EEG files: 340 (cache=318, missing=23)
[Info] Loaded pretrained CBraMod weights: /home/RA/EEG_Challenge/Challenge2/CBraMod/pretrained_weights/pretrained_weights.pth


[Ep01] TrainLoss=0.4153 | Val MSE=0.4570 MAE=0.5244 R2=-1.411 time=2169.4s


[Ep02] TrainLoss=0.3998 | Val MSE=0.4666 MAE=0.5379 R2=-1.462 time=2240.9s


[Ep03] TrainLoss=0.3889 | Val MSE=0.4360 MAE=0.4974 R2=-1.300 time=2308.4s


[Ep04] TrainLoss=0.4003 | Val MSE=0.4535 MAE=0.5114 R2=-1.393 time=2190.1s


...

[Ep06] TrainLoss=0.3896 | Val MSE=0.4701 MAE=0.5266 R2=-1.480 time=2153.8s


[Ep07] TrainLoss=0.3843 | Val MSE=0.4987 MAE=0.5581 R2=-1.631 time=2309.4s


[Ep08] TrainLoss=0.3817 | Val MSE=0.4721 MAE=0.5289 R2=-1.491 time=2279.5s


[Ep09] TrainLoss=0.3926 | Val MSE=0.4511 MAE=0.5114 R2=-1.380 time=2155.4s


[Ep10] TrainLoss=0.3863 | Val MSE=0.4866 MAE=0.5324 R2=-1.567 time=2152.7s
✅ Best: {'NRMSE': 0.4360489249229431, 'epoch': 3}


In [4]:
import os, random, time, warnings, json, re, glob
import numpy as np, pandas as pd, torch, mne
import torch.nn as nn
import torch.nn.functional as F
from torch.utils.data import Dataset, DataLoader
from tqdm import tqdm

# ============================================================
# CONFIG
# ============================================================
BIDS_ROOT   = "/data5/open_data/HBN/EEG_BIDS"
CACHE_DIR = "/data5/open_data/HBN/Preprocessed_EEG/0922try_bySubject"
# MISSING_TXT = "/home/RA/EEG_Challenge/Challenge2/logs/missing_cache_files.txt"
CBRAMOD_WEIGHTS = "/home/RA/EEG_Challenge/Challenge2/CBraMod/pretrained_weights/pretrained_weights.pth"

DEVICE = "cuda" if torch.cuda.is_available() else "cpu"
SFREQ  = 100
CROP_T = 1000
N_CHANS = 128   # Cz 제거 후
BATCH   = 8
EPOCHS  = 10
LR_FRONT, LR_BACKB, LR_HEAD = 3e-4, 1e-4, 3e-4
WD = 1e-2
SEED = 42

VAL_RELEASE = "R5"

VALID_SUBJECT_CSV = "/home/RA/EEG_Challenge/Challenge2/valid_subjects_with_release.csv"

warnings.filterwarnings("ignore")
mne.set_log_level("CRITICAL")
random.seed(SEED); np.random.seed(SEED); torch.manual_seed(SEED); torch.cuda.manual_seed_all(SEED)

# ============================================================
# 1. 파일 리스트 구성
# ============================================================
def build_raw_filelist(bids_root, subjects_to_find):
    """
    BIDS_ROOT를 스캔하여 subjects_to_find 목록에 있는
    대상자들의 모든 .set 파일 경로를 찾습니다.
    (subjects_to_find는 set이나 list 형태)
    """
    eeg_files = []
    subject_set = set(subjects_to_find)
    print(f"Building raw .set file list for {len(subject_set)} subjects...")
    
    # BIDS_ROOT/ds*/sub-*/eeg/*.set 패턴 검색
    search_pattern = os.path.join(bids_root, "ds*", "sub-*", "eeg", "*.set")
    all_set_files = glob.glob(search_pattern)
    
    if not all_set_files:
        print(f"⚠️  경고: '{search_pattern}' 경로에서 .set 파일을 찾을 수 없습니다.")
        return []

    # 파일 경로에서 'sub-XXXX' 추출
    subject_pattern = re.compile(r'(sub-[a-zA-Z0-9]+)')

    for f_path in tqdm(all_set_files, desc="Scanning .set files"):
        match = subject_pattern.search(f_path)
        if match:
            subj_id = match.group(1)
            if subj_id in subject_set:
                eeg_files.append(f_path)
                
    print(f"✅ Found {len(eeg_files)} raw .set files for the valid subjects.")
    return sorted(list(set(eeg_files)))


# ============================================================
# 2. 메타데이터 로드
# ============================================================
def load_participants(bids_root):
    dfs=[]
    for ds in sorted(os.listdir(bids_root)):
        pfile=os.path.join(bids_root, ds, "participants.tsv")
        if os.path.exists(pfile):
            df=pd.read_csv(pfile,sep="\t")
            df["release_number"]=ds ##########
            dfs.append(df)
    meta=pd.concat(dfs,ignore_index=True)
    meta.columns=[c.lower() for c in meta.columns]
    meta=meta.drop_duplicates(subset=["participant_id"]).set_index("participant_id")

    # p_factor, age, sex, ehq_total 정리
    if "p_factor" not in meta.columns:
        alt=[c for c in meta.columns if "p" in c and "factor" in c]
        if alt: meta=meta.rename(columns={alt[0]:"p_factor"})
    meta["sex"]=meta["sex"].map({"M":1,"F":0,"m":1,"f":0}).fillna(0.5)
    for c in ["age","ehq_total"]:
        if c not in meta.columns: meta[c]=np.nan
        meta[c]=meta[c].fillna(meta[c].mean())
        m,s=meta[c].mean(),meta[c].std()+1e-8
        meta[c]=(meta[c]-m)/s
    return meta

# ============================================================
# 3. Dataset
# ============================================================
def subj_from_fname(fname):
    base=os.path.basename(fname)
    return base.split("_")[0]

class RawEEGDataset(Dataset):
    def __init__(self, files, meta, train=True):
        self.files=files; self.meta=meta; self.train=train
        self.targets=meta["p_factor"].dropna()
        self.subj_ids=set(self.targets.index)

    def __len__(self): return len(self.files)

    def __getitem__(self,i):
        path=self.files[i]
        subj=subj_from_fname(path)
        if subj not in self.subj_ids:
            return torch.zeros((N_CHANS,CROP_T)), torch.tensor(0.), torch.zeros(3)
        raw=mne.io.read_raw_eeglab(path,preload=True,verbose=False)
        raw.filter(0.5,50.,fir_design="firwin")
        raw.set_eeg_reference("average",projection=False)
        raw.resample(SFREQ)
        x=raw.get_data()
        if x.shape[0]==129: x=x[:-1,:]  # Cz 제거
        x=(x-x.mean(axis=1,keepdims=True))/(x.std(axis=1,keepdims=True)+1e-6)

        T=x.shape[1]
        if T<CROP_T:
            pad=np.zeros((x.shape[0],CROP_T)); pad[:,:T]=x; x=pad
        else:
            st=np.random.randint(0,T-CROP_T+1) if self.train else (T-CROP_T)//2
            x=x[:,st:st+CROP_T]

        y=float(self.meta.loc[subj,"p_factor"]) if subj in self.meta.index else 0.
        meta_vec=torch.tensor([
            self.meta.loc[subj,"age"],
            self.meta.loc[subj,"sex"],
            self.meta.loc[subj,"ehq_total"]
        ],dtype=torch.float32) if subj in self.meta.index else torch.zeros(3)
        return torch.tensor(x,dtype=torch.float32), torch.tensor(y,dtype=torch.float32), meta_vec

# ============================================================
# 4. 모델 정의 (CBraMod + regression head)
# ============================================================
class SincConv1d(nn.Module):
    def __init__(self, out_channels=64, kernel_size=129, sample_rate=100, min_hz=0.3, max_hz=45.0):
        super().__init__()
        self.out_channels = out_channels
        self.kernel_size  = kernel_size
        self.sample_rate  = sample_rate
        self.min_hz       = float(min_hz)
        self.max_hz       = float(max_hz)
        low  = torch.linspace(self.min_hz, self.max_hz - 5.0, out_channels)
        band = torch.ones(out_channels) * 5.0
        self.low_hz_  = nn.Parameter(low)
        self.band_hz_ = nn.Parameter(band)
        n = torch.arange(-(kernel_size // 2), kernel_size // 2 + 1).float()
        self.register_buffer("n", n)

    def forward(self, x):  # x: (B, C, T)
        B, C, T = x.shape
        device  = x.device
        dtype   = x.dtype

        # low: (out,)
        low  = torch.clamp(torch.abs(self.low_hz_), min=self.min_hz, max=self.max_hz - 1.0)
        # high: clamp with tensor min/max (same shape)
        raw_high = low + torch.abs(self.band_hz_)
        min_v    = low + 1.0
        max_v    = torch.full_like(low, self.max_hz)
        high     = torch.clamp(raw_high, min=min_v, max=max_v)

        n = self.n.to(device=device, dtype=dtype)
        window = torch.hamming_window(self.kernel_size, periodic=False, dtype=dtype, device=device)

        filters = []
        nyq = (self.sample_rate / 2.0)
        for i in range(self.out_channels):
            f1 = low[i]  / nyq
            f2 = high[i] / nyq
            h1 = 2 * f2 * torch.sinc(2 * f2 * n)
            h2 = 2 * f1 * torch.sinc(2 * f1 * n)
            bandpass = (h1 - h2) * window
            filters.append(bandpass)
        filt = torch.stack(filters, dim=0).unsqueeze(1)  # (out, 1, K)

        x_dw = x.view(B * C, 1, T)
        y = F.conv1d(x, filt, stride=1, padding=self.kernel_size // 2)

        y = y.view(B, C, self.out_channels, y.shape[-1]).sum(dim=1)  # (B, out, T)
        return y


class SEBlock(nn.Module):
    def __init__(self,c,r=8):
        super().__init__()
        self.fc1=nn.Linear(c,c//r); self.fc2=nn.Linear(c//r,c)
    def forward(self,x):
        s=x.mean(-1); e=F.relu(self.fc1(s)); e=torch.sigmoid(self.fc2(e)).unsqueeze(-1)
        return x*e

class CBraModBackbone(nn.Module):
    def __init__(self,out_dim=512):
        super().__init__()
        self.conv=nn.Sequential(
            nn.Conv1d(64,128,7,padding=3),nn.ReLU(),
            nn.Conv1d(128,256,5,padding=2),nn.ReLU(),
            nn.AdaptiveAvgPool1d(1))
        self.fc=nn.Linear(256,out_dim); self.out_dim=out_dim
    def load_weights(self,wpath):
        sd=torch.load(wpath,map_location="cpu")
        self.load_state_dict(sd,strict=False)
        print(f"[Info] Loaded pretrained CBraMod weights: {wpath}")
    def forward(self,x):
        h=self.conv(x).squeeze(-1)
        return self.fc(h)

class RegressionHead(nn.Module):
    def __init__(self,in_dim,meta_dim=3,hetero=True):
        super().__init__()
        self.hetero=hetero
        self.meta_fc=nn.Sequential(nn.Linear(meta_dim,32),nn.ReLU())
        self.fc=nn.Sequential(nn.Linear(in_dim+32,128),nn.ReLU(),nn.Linear(128,1))
        if hetero:
            self.logv=nn.Sequential(nn.Linear(in_dim+32,64),nn.ReLU(),nn.Linear(64,1))
    def forward(self,f,meta):
        
        m=self.meta_fc(meta)
        z=torch.cat([f,m],1)
        
        mu=self.fc(z).squeeze(-1)
        if self.hetero:
            lv = self.logv(z)
            
            if isinstance(lv, (tuple, list)):
                lv = lv[0]
            logv = lv.squeeze(-1)
            logv = torch.clamp(logv, min=-6.0, max=6.0)

            return mu,logv

def hetero_nll(y,mu,logv):
    inv=torch.exp(-logv)
    return 0.5*((y-mu)**2*inv+logv).mean()

# ============================================================
# 5. 학습 루프
# ============================================================
def train_epoch(model,head,loader,opt,scaler,hetero):
    model.train(); head.train(); total=0
    for x,y,meta in tqdm(loader,desc="Train",leave=False):
        x,y,meta=x.to(DEVICE),y.to(DEVICE),meta.to(DEVICE)
        opt.zero_grad(set_to_none=True)
        with torch.cuda.amp.autocast(True):
            z=model(x); 
            out=head(z,meta)
            if hetero:
                mu,logv=out; loss=hetero_nll(y,mu,logv)
            else:
                mu=out; loss=F.mse_loss(mu,y)
        scaler.scale(loss).backward(); scaler.step(opt); scaler.update()
        total+=loss.item()
    return total/len(loader)

@torch.no_grad()
def evaluate(model,head,loader,hetero):
    model.eval(); head.eval()
    preds,trues=[],[]
    for x,y,meta in tqdm(loader,desc="Val",leave=False):
        x,meta=x.to(DEVICE),meta.to(DEVICE)
        out=head(model(x),meta)
        mu=out[0] if hetero else out
        preds.extend(mu.detach().cpu().numpy()); trues.extend(y.numpy())
    preds,trues=np.array(preds),np.array(trues)
    mae=np.mean(np.abs(preds-trues))
    mse=np.mean((preds-trues)**2)
    r2=1-np.sum((preds-trues)**2)/np.sum((trues-trues.mean())**2)
    nrmse=np.sqrt(mse)/(trues.std()+1e-8)
    return mae,mse,r2,nrmse

# ============================================================
# 6. 실행
# ============================================================
def main():
    # 1. 모든 참가자 메타데이터 로드
    meta = load_participants(BIDS_ROOT)

    # 2. 챌린지 2용 유효 대상자 목록(CSV) 로드
    if not os.path.exists(VALID_SUBJECT_CSV):
        raise FileNotFoundError(f"유효 대상자 CSV 파일을 찾을 수 없습니다: {VALID_SUBJECT_CSV}")
    
    valid_df = pd.read_csv(VALID_SUBJECT_CSV)
    valid_subject_ids = set(valid_df['participant_id'])
    print(f"Loaded {len(valid_subject_ids)} valid subject IDs from {VALID_SUBJECT_CSV}.")

    # 3. 메타데이터 필터링
    #    (CSV 목록에 있고 P-factor 점수도 있는 대상자만 선택)
    meta = meta.reindex(valid_subject_ids) # CSV 기준으로 인덱스 재설정
    meta = meta[meta['p_factor'].notna()]  # P-factor 없는 대상자 제거
    print(f"Filtered to {len(meta.index)} subjects with valid demo + p_factor.")

    # 4. Train/Val 대상자 ID 분리
    val_subjects = set(meta[meta['release_number'] == VAL_RELEASE].index)
    train_subjects = set(meta.index) - val_subjects
    print(f"Train subjects: {len(train_subjects)}, Val subjects: {len(val_subjects)}")

    # 5. 대상자에 해당하는 .set 파일 리스트 생성
    all_files = build_raw_filelist(BIDS_ROOT, meta.index) # meta.index = train + val

    # 6. 파일을 Train/Val로 분리
    train_files = [f for f in all_files if subj_from_fname(f) in train_subjects]
    val_files = [f for f in all_files if subj_from_fname(f) in val_subjects]
    print(f"Total files: {len(all_files)} (Train: {len(train_files)}, Val: {len(val_files)})")

    # 7. Dataset 및 DataLoader 생성 (RawEEGDataset 사용)
    train_ds = RawEEGDataset(train_files, meta, train=True)
    val_ds   = RawEEGDataset(val_files, meta, train=False)
    train_loader = DataLoader(train_ds, batch_size=BATCH, shuffle=True, num_workers=4)
    val_loader   = DataLoader(val_ds, batch_size=BATCH, shuffle=False, num_workers=2)

    # Model
    front=SincConv1d(out_channels=64,kernel_size=129,sample_rate=SFREQ)
    se=SEBlock(64); backbone=CBraModBackbone(out_dim=512)
    backbone.load_weights(CBRAMOD_WEIGHTS)
    model=nn.Sequential(front,se,backbone).to(DEVICE)
    head=RegressionHead(512,meta_dim=3,hetero=True).to(DEVICE)
    opt=torch.optim.AdamW(list(model.parameters())+list(head.parameters()),
                          lr=3e-4,weight_decay=WD)
    scaler=torch.cuda.amp.GradScaler()
    best={"NRMSE":9e9}

    for ep in range(1, EPOCHS + 1):
        t0 = time.time()
        tr = train_epoch(model, head, train_loader, opt, scaler, hetero=True)
        mae, mse, r2, nrmse = evaluate(model, head, val_loader, hetero=True)
        print(f"[Ep{ep:02d}] TrainLoss={tr:.4f} | Val MSE={mse:.4f} MAE={mae:.4f} R2={r2:.3f} time={time.time()-t0:.1f}s")

        # ✅ MSE 기준으로 저장 (NRMSE 제외)
        if mse < best["NRMSE"]:   # key 이름 그대로 두되 값은 MSE
            best = {"NRMSE": float(mse), "epoch": ep}  # float32 → float 변환
            torch.save({
                "front": front.state_dict(),
                "backbone": backbone.state_dict(),
                "head": head.state_dict()
            }, "best_cbramod_raw_finetune.pth")

            # ✅ JSON 직렬화 가능한 타입으로 기록
            with open("best_metrics.json", "w") as f:
                json.dump({
                    "MAE": float(mae),
                    "MSE": float(mse),
                    "R2": float(r2)
                }, f, indent=2)

    print("✅ Best:", best)


if __name__=="__main__":
    main()


Loaded 2844 valid subject IDs from /home/RA/EEG_Challenge/Challenge2/valid_subjects_with_release.csv.
Filtered to 2810 subjects with valid demo + p_factor.
Train subjects: 2810, Val subjects: 0
Building raw .set file list for 2810 subjects...


Scanning .set files: 100%|██████████| 26672/26672 [00:00<00:00, 700552.18it/s]


✅ Found 25371 raw .set files for the valid subjects.
Total files: 25371 (Train: 25371, Val: 0)
[Info] Loaded pretrained CBraMod weights: /home/RA/EEG_Challenge/Challenge2/CBraMod/pretrained_weights/pretrained_weights.pth


RuntimeError: Given groups=1, weight of size [64, 1, 129], expected input[8, 128, 1000] to have 1 channels, but got 128 channels instead

: 

샘플링 버전

In [1]:
%env CUDA_VISIBLE_DEVICES=1

env: CUDA_VISIBLE_DEVICES=1


In [4]:
import os, random, time, warnings, json, re, glob
import numpy as np, pandas as pd, torch, mne
import torch.nn as nn
import torch.nn.functional as F
from torch.utils.data import Dataset, DataLoader
from tqdm import tqdm

# ============================================================
# CONFIG
# ============================================================
BIDS_ROOT   = "/data5/open_data/HBN/EEG_BIDS"
CACHE_DIR = "/data5/open_data/HBN/Preprocessed_EEG/0922try_bySubject" # ✅ bySubject 구조
CBRAMOD_WEIGHTS = "/home/RA/EEG_Challenge/Challenge2/CBraMod/pretrained_weights/pretrained_weights.pth"

DEVICE = "cuda" if torch.cuda.is_available() else "cpu"
SFREQ  = 100
CROP_T = 1000
N_CHANS = 128   # Cz 제거 후
BATCH   = 8
EPOCHS  = 10
LR_FRONT, LR_BACKB, LR_HEAD = 3e-4, 1e-4, 3e-4
WD = 1e-2
SEED = 42

VAL_RELEASE = "R5"
VALID_SUBJECT_CSV = "/home/RA/EEG_Challenge/Challenge2/valid_subjects_with_release.csv"

warnings.filterwarnings("ignore")
mne.set_log_level("CRITICAL")
random.seed(SEED); np.random.seed(SEED); torch.manual_seed(SEED); torch.cuda.manual_seed_all(SEED)

# ============================================================
# 1. 파일 리스트 구성 (⭐️ 수정된 섹션 ⭐️)
# ============================================================
def build_cached_filelist(cache_dir, subjects_to_find):
    """
    CACHE_DIR (bySubject)을 스캔하여 subjects_to_find 목록에 있는
    대상자들의 모든 .set 파일 경로를 찾습니다.
    구조: cache_dir/sub-XXXX/**/*.set (하위 폴더 포함)
    """
    eeg_files = []
    subject_set = set(subjects_to_find)
    print(f"Building cached .set file list from {cache_dir} (bySubject structure)...")
    
    if not os.path.exists(cache_dir):
        print(f"❌ Error: Cache directory not found: {cache_dir}")
        return []
        
    # subjects_to_find 목록에 있는 각 피험자에 대해 폴더를 직접 탐색
    for subj_id in tqdm(subject_set, desc="Scanning subject folders"):
        subj_folder = os.path.join(cache_dir, subj_id)
        
        if not os.path.exists(subj_folder):
            continue
            
        # 해당 피험자 폴더 및 그 하위 폴더 내의 모든 .set 파일 검색
        search_pattern = os.path.join(subj_folder, "**", "*.set")
        files_in_folder = glob.glob(search_pattern, recursive=True)
        
        eeg_files.extend(files_in_folder)
                
    if not eeg_files:
        print(f"⚠️  경고: '{cache_dir}/sub-*/.../*.set' 경로에서 .set 파일을 찾을 수 없습니다.")
        return []

    print(f"✅ Found {len(eeg_files)} cached .set files for the valid subjects.")
    return sorted(list(set(eeg_files)))


# ============================================================
# 2. 메타데이터 로드
# ============================================================
def load_participants(bids_root):
    dfs=[]
    for ds in sorted(os.listdir(bids_root)):
        pfile=os.path.join(bids_root, ds, "participants.tsv")
        if os.path.exists(pfile):
            df=pd.read_csv(pfile,sep="\t")
            df["release_number"]=ds ##########
            dfs.append(df)
    meta=pd.concat(dfs,ignore_index=True)
    meta.columns=[c.lower() for c in meta.columns]
    meta=meta.drop_duplicates(subset=["participant_id"]).set_index("participant_id")

    # p_factor, age, sex, ehq_total 정리
    if "p_factor" not in meta.columns:
        alt=[c for c in meta.columns if "p" in c and "factor" in c]
        if alt: meta=meta.rename(columns={alt[0]:"p_factor"})
    meta["sex"]=meta["sex"].map({"M":1,"F":0,"m":1,"f":0}).fillna(0.5)
    for c in ["age","ehq_total"]:
        if c not in meta.columns: meta[c]=np.nan
        meta[c]=meta[c].fillna(meta[c].mean())
        m,s=meta[c].mean(),meta[c].std()+1e-8
        meta[c]=(meta[c]-m)/s
    return meta

# ============================================================
# 3. Dataset
# ============================================================
def subj_from_cached_fname(fname):
    base=os.path.basename(fname)
    return base.split("_")[0]

class CachedEEGDataset(Dataset):
    """
    미리 전처리(리샘플링 등)된 .set 파일을 로드하는 Dataset.
    파일이 이미 100Hz이며, Cz가 129번째 채널에 있다고 가정합니다.
    """
    def __init__(self, files, meta, train=True):
        self.files = files
        self.meta = meta
        self.train = train
        self.targets = meta["p_factor"].dropna()
        self.subj_ids = set(self.targets.index)

    def __len__(self): 
        return len(self.files)

    def __getitem__(self, i):
        path = self.files[i]
        subj = subj_from_cached_fname(path)
        
        if subj not in self.subj_ids:
            return torch.zeros((N_CHANS, CROP_T)), torch.tensor(0.), torch.zeros(3)

        # 1. 미리 캐시된 .set 파일 로드
        try:
            # 가정: 이 .set 파일은 이미 100Hz, 필터링, 레퍼런싱이 완료됨
            raw = mne.io.read_raw_eeglab(path, preload=True, verbose=False)
        except Exception as e:
            print(f"Warning: Error loading {path}: {e}. Returning zeros.")
            return torch.zeros((N_CHANS, CROP_T)), torch.tensor(0.), torch.zeros(3)

        # 2. 데이터 추출
        x = raw.get_data().astype(np.float32)
        
        # 3. Cz 제거 (129 채널일 경우)
        if x.shape[0] == 129:
            x = x[:-1, :]
        
        # 4. 채널별 Z-정규화
        x = (x - x.mean(axis=1, keepdims=True)) / (x.std(axis=1, keepdims=True) + 1e-6)

        # 5. 윈도우잉 (Crop)
        T = x.shape[1]
        if T < CROP_T:
            pad = np.zeros((N_CHANS, CROP_T), dtype=np.float32)
            pad[:, :T] = x
            x = pad
        else:
            # Train: 랜덤 크롭, Val: 센터 크롭
            st = np.random.randint(0, T - CROP_T + 1) if self.train else (T - CROP_T) // 2
            x = x[:, st : st + CROP_T]

        # 6. 레이블 및 메타데이터 가져오기
        y = float(self.meta.loc[subj, "p_factor"])
        meta_vec = torch.tensor([
            self.meta.loc[subj, "age"],
            self.meta.loc[subj, "sex"],
            self.meta.loc[subj, "ehq_total"]
        ], dtype=torch.float32)
        
        return torch.tensor(x), torch.tensor(y, dtype=torch.float32), meta_vec

# ============================================================
# 4. 모델 정의 (CBraMod + regression head)
# ============================================================
class SincConv1d(nn.Module):
    def __init__(self, out_channels=64, kernel_size=129, sample_rate=100, min_hz=0.3, max_hz=45.0):
        super().__init__()
        self.out_channels = out_channels
        self.kernel_size  = kernel_size
        self.sample_rate  = sample_rate
        self.min_hz       = float(min_hz)
        self.max_hz       = float(max_hz)
        low  = torch.linspace(self.min_hz, self.max_hz - 5.0, out_channels)
        band = torch.ones(out_channels) * 5.0
        self.low_hz_  = nn.Parameter(low)
        self.band_hz_ = nn.Parameter(band)
        n = torch.arange(-(kernel_size // 2), kernel_size // 2 + 1).float()
        self.register_buffer("n", n)

    def forward(self, x):  # x: (B, C, T)
        B, C, T = x.shape
        device  = x.device
        dtype   = x.dtype

        # ... (low, high, window, filters 계산) ...
        low  = torch.clamp(torch.abs(self.low_hz_), min=self.min_hz, max=self.max_hz - 1.0)
        raw_high = low + torch.abs(self.band_hz_)
        min_v    = low + 1.0
        max_v    = torch.full_like(low, self.max_hz)
        high     = torch.clamp(raw_high, min=min_v, max=max_v)
        n = self.n.to(device=device, dtype=dtype)
        window = torch.hamming_window(self.kernel_size, periodic=False, dtype=dtype, device=device)
        filters = []
        nyq = (self.sample_rate / 2.0)
        for i in range(self.out_channels):
            f1 = low[i]  / nyq
            f2 = high[i] / nyq
            h1 = 2 * f2 * torch.sinc(2 * f2 * n)
            h2 = 2 * f1 * torch.sinc(2 * f1 * n)
            bandpass = (h1 - h2) * window
            filters.append(bandpass)
        filt = torch.stack(filters, dim=0).unsqueeze(1)  # (out, 1, K)

        x_dw = x.view(B * C, 1, T)
        
        # [ ⭐️ 수정됨 ⭐️ ] x 대신 x_dw 를 입력으로 사용
        y = F.conv1d(x_dw, filt, stride=1, padding=self.kernel_size // 2)

        y = y.view(B, C, self.out_channels, y.shape[-1]).sum(dim=1)  # (B, out, T)
        return y


class SEBlock(nn.Module):
    def __init__(self,c,r=8):
        super().__init__()
        self.fc1=nn.Linear(c,c//r); self.fc2=nn.Linear(c//r,c)
    def forward(self,x):
        s=x.mean(-1); e=F.relu(self.fc1(s)); e=torch.sigmoid(self.fc2(e)).unsqueeze(-1)
        return x*e

class CBraModBackbone(nn.Module):
    def __init__(self,out_dim=512):
        super().__init__()
        self.conv=nn.Sequential(
            nn.Conv1d(64,128,7,padding=3),nn.ReLU(),
            nn.Conv1d(128,256,5,padding=2),nn.ReLU(),
            nn.AdaptiveAvgPool1d(1))
        self.fc=nn.Linear(256,out_dim); self.out_dim=out_dim
    def load_weights(self,wpath):
        sd=torch.load(wpath,map_location="cpu")
        self.load_state_dict(sd,strict=False)
        print(f"[Info] Loaded pretrained CBraMod weights: {wpath}")
    def forward(self,x):
        h=self.conv(x).squeeze(-1)
        return self.fc(h)

class RegressionHead(nn.Module):
    def __init__(self,in_dim,meta_dim=3,hetero=True):
        super().__init__()
        self.hetero=hetero
        self.meta_fc=nn.Sequential(nn.Linear(meta_dim,32),nn.ReLU())
        self.fc=nn.Sequential(nn.Linear(in_dim+32,128),nn.ReLU(),nn.Linear(128,1))
        if hetero:
            self.logv=nn.Sequential(nn.Linear(in_dim+32,64),nn.ReLU(),nn.Linear(64,1))
    def forward(self,f,meta):
        
        m=self.meta_fc(meta)
        z=torch.cat([f,m],1)
        
        mu=self.fc(z).squeeze(-1)
        if self.hetero:
            lv = self.logv(z)
            
            if isinstance(lv, (tuple, list)):
                lv = lv[0]
            logv = lv.squeeze(-1)
            logv = torch.clamp(logv, min=-6.0, max=6.0)

            return mu,logv

def hetero_nll(y,mu,logv):
    inv=torch.exp(-logv)
    return 0.5*((y-mu)**2*inv+logv).mean()

# ============================================================
# 5. 학습 루프
# ============================================================
def train_epoch(model,head,loader,opt,scaler,hetero):
    model.train(); head.train(); total=0
    for x,y,meta in tqdm(loader,desc="Train",leave=False):
        x,y,meta=x.to(DEVICE),y.to(DEVICE),meta.to(DEVICE)
        opt.zero_grad(set_to_none=True)
        with torch.cuda.amp.autocast(True):
            z=model(x); 
            out=head(z,meta)
            if hetero:
                mu,logv=out; loss=hetero_nll(y,mu,logv)
            else:
                mu=out; loss=F.mse_loss(mu,y)
        scaler.scale(loss).backward(); scaler.step(opt); scaler.update()
        total+=loss.item()
    return total/len(loader)

@torch.no_grad()
def evaluate(model,head,loader,hetero):
    model.eval(); head.eval()
    preds,trues=[],[]
    for x,y,meta in tqdm(loader,desc="Val",leave=False):
        x,meta=x.to(DEVICE),meta.to(DEVICE)
        out=head(model(x),meta)
        mu=out[0] if hetero else out
        preds.extend(mu.detach().cpu().numpy()); trues.extend(y.numpy())
    preds,trues=np.array(preds),np.array(trues)
    mae=np.mean(np.abs(preds-trues))
    mse=np.mean((preds-trues)**2)
    r2=1-np.sum((preds-trues)**2)/np.sum((trues-trues.mean())**2)
    nrmse=np.sqrt(mse)/(trues.std()+1e-8)
    return mae,mse,r2,nrmse

# ============================================================
# 6. 실행 (Train/Val 분리 로직 수정)
# ============================================================
def main():
    # 1. 모든 참가자 메타데이터 로드
    meta = load_participants(BIDS_ROOT) # 'release_number' ('ds...') 포함

    # 2. 챌린지 2용 유효 대상자 목록(CSV) 로드
    if not os.path.exists(VALID_SUBJECT_CSV):
        raise FileNotFoundError(f"유효 대상자 CSV 파일을 찾을 수 없습니다: {VALID_SUBJECT_CSV}")
    
    valid_df = pd.read_csv(VALID_SUBJECT_CSV)
    valid_subject_ids = set(valid_df['participant_id'])
    print(f"Loaded {len(valid_subject_ids)} valid subject IDs from {VALID_SUBJECT_CSV}.")

    # --- [수정된 로직] ---
    
    # 3. CSV의 'release_number' 컬럼명을 'release'로 변경 (충돌 회피)
    if 'release_number' in valid_df.columns:
        valid_df = valid_df.rename(columns={'release_number': 'release'})
    elif 'release' not in valid_df.columns:
        raise ValueError(f"CSV 파일에 'release' 또는 'release_number' 컬럼이 없습니다.")
    
    # 'release' 컬럼('R5' 등)을 'meta' DataFrame에 병합
    valid_df_subset = valid_df[['participant_id', 'release']].set_index('participant_id')
    meta = meta.join(valid_df_subset) # 'release' 컬럼 추가

    # 4. 메타데이터 필터링
    meta = meta.reindex(valid_subject_ids) # CSV 목록 기준으로 필터링
    meta = meta[meta['p_factor'].notna()]  # P-factor 없는 대상자 제거
    
    if VAL_RELEASE in meta['release'].values:
        print(f"✅ Successfully joined 'release' column (found '{VAL_RELEASE}').")
    else:
        print(f"⚠️  WARNING: Could not find '{VAL_RELEASE}' in 'release' column after join.")
        
    print(f"Filtered to {len(meta.index)} subjects with valid demo + p_factor.")

    # 5. Train/Val 대상자 ID 분리
    #    'release' (R5) 또는 'release_number' (ds005510) 확인
    val_mask = (meta['release'] == VAL_RELEASE) | (meta['release_number'] == 'ds005510')
    val_subjects = set(meta[val_mask].index)
    train_subjects = set(meta.index) - val_subjects
    
    # --- [수정 끝] ---
    
    print(f"Train subjects: {len(train_subjects)}, Val subjects: {len(val_subjects)}")

    # 6. 대상자에 해당하는 .set 파일 리스트 생성 (수정된 build_cached_filelist 호출)
    all_files = build_cached_filelist(CACHE_DIR, meta.index) 

    # 7. 파일을 Train/Val로 분리
    train_files = [f for f in all_files if subj_from_cached_fname(f) in train_subjects]
    val_files = [f for f in all_files if subj_from_cached_fname(f) in val_subjects]
    
    if len(val_subjects) == 0 or len(val_files) == 0:
        print("\n" + "="*50)
        print("❌ CRITICAL ERROR: Validation set is empty.")
        print(" 'release' 또는 'release_number' 컬럼 매칭에 실패했습니다.")
        if 'release' in meta.columns:
            print(meta[['release', 'release_number']].value_counts())
        else:
            print(meta[['release_number']].value_counts())
        print("="*50 + "\n")
        raise ValueError("Validation set construction failed.")
        
    print(f"Total files: {len(all_files)} (Train: {len(train_files)}, Val: {len(val_files)})")

    # 8. Dataset 및 DataLoader 생성 (CachedEEGDataset 사용)
    train_ds = CachedEEGDataset(train_files, meta, train=True)
    val_ds   = CachedEEGDataset(val_files, meta, train=False)
    train_loader = DataLoader(train_ds, batch_size=BATCH, shuffle=True, num_workers=4)
    val_loader   = DataLoader(val_ds, batch_size=BATCH, shuffle=False, num_workers=2)

    # 9. 모델 및 학습 (변경 없음)
    front=SincConv1d(out_channels=64,kernel_size=129,sample_rate=SFREQ)
    se=SEBlock(64); backbone=CBraModBackbone(out_dim=512)
    backbone.load_weights(CBRAMOD_WEIGHTS)
    model=nn.Sequential(front,se,backbone).to(DEVICE)
    head=RegressionHead(512,meta_dim=3,hetero=True).to(DEVICE)
    opt=torch.optim.AdamW(list(model.parameters())+list(head.parameters()),
                          lr=3e-4,weight_decay=WD)
    scaler=torch.cuda.amp.GradScaler()
    best={"NRMSE":9e9}

    for ep in range(1, EPOCHS + 1):
        t0 = time.time()
        tr = train_epoch(model, head, train_loader, opt, scaler, hetero=True)
        mae, mse, r2, nrmse = evaluate(model, head, val_loader, hetero=True)
        print(f"[Ep{ep:02d}] TrainLoss={tr:.4f} | Val MSE={mse:.4f} MAE={mae:.4f} R2={r2:.3f} time={time.time()-t0:.1f}s")

        if mse < best["NRMSE"]:
            best = {"NRMSE": float(mse), "epoch": ep}
            torch.save({
                "front": front.state_dict(),
                "backbone": backbone.state_dict(),
                "head": head.state_dict()
            }, "best_cbramod_cached_finetune.pth")

            with open("best_metrics_cached.json", "w") as f:
                json.dump({
                    "MAE": float(mae),
                    "MSE": float(mse),
                    "R2": float(r2)
                }, f, indent=2)

    print("✅ Best:", best)


if __name__=="__main__":
    main()


Loaded 2844 valid subject IDs from /home/RA/EEG_Challenge/Challenge2/valid_subjects_with_release.csv.
✅ Successfully joined 'release' column (found 'R5').
Filtered to 2810 subjects with valid demo + p_factor.
Train subjects: 2375, Val subjects: 435
Building cached .set file list from /data5/open_data/HBN/Preprocessed_EEG/0922try_bySubject (bySubject structure)...


Scanning subject folders: 100%|██████████| 2810/2810 [01:16<00:00, 36.80it/s] 


✅ Found 24658 cached .set files for the valid subjects.
Total files: 24658 (Train: 20387, Val: 4271)
[Info] Loaded pretrained CBraMod weights: /home/RA/EEG_Challenge/Challenge2/CBraMod/pretrained_weights/pretrained_weights.pth


[Ep01] TrainLoss=0.3999 | Val MSE=0.7606 MAE=0.7048 R2=-0.000 time=6038.5s


[Ep02] TrainLoss=0.3963 | Val MSE=0.7536 MAE=0.6994 R2=0.009 time=2247.2s


...

[Ep06] TrainLoss=0.3885 | Val MSE=0.7710 MAE=0.7077 R2=-0.014 time=1449.6s


[Ep07] TrainLoss=0.3871 | Val MSE=0.7661 MAE=0.7055 R2=-0.007 time=1487.9s


[Ep08] TrainLoss=0.3847 | Val MSE=0.7643 MAE=0.7041 R2=-0.005 time=1566.0s


[Ep09] TrainLoss=0.3830 | Val MSE=0.7715 MAE=0.7082 R2=-0.014 time=1603.7s


[Ep10] TrainLoss=0.3813 | Val MSE=0.7744 MAE=0.7110 R2=-0.018 time=1580.8s
✅ Best: {'NRMSE': 0.7535700798034668, 'epoch': 2}


In [7]:
# ============================================================
# Challenge 2 - Fast Hierarchical CBraMod (no cross-task transfer)
# 1) Raw EEG -> optional .npy cache (speed)
# 2) Preproc: 0.5-45 Hz, CAR, resample, Cz drop -> zscore
# 3) Multi-band fixed FIR bank (+ learnable SincConv optional)
# 4) Window encoder (CBraMod-style conv)
# 5) Subject-level Transformer head (K windows per subject)
# 6) Meta conditioning + Multi-task regression (p/int/ext)
# 7) Loss: MSE + 0.2*CorrLoss  (CCC/NMSE 제거)
# 8) Light aug: jitter + time-mask + mixup
# ============================================================

import os, random, time, warnings, json
from collections import defaultdict
import numpy as np
import pandas as pd
import mne
from tqdm import tqdm

import torch
import torch.nn as nn
import torch.nn.functional as F
from torch.utils.data import Dataset, DataLoader

# ---------------- config ----------------
BIDS_ROOT   = "/data5/open_data/HBN/EEG_BIDS"
CACHE_DIR   = "/data5/open_data/HBN/cache_eeg_windows"   # 기존 캐시 파일명 참고용(optional)
NPY_CACHE   = "/data5/open_data/HBN/cache_eeg_raw_npy"   # 새로 생성할 .npy 캐시
MISSING_TXT = "/home/RA/EEG_Challenge/Challenge2/logs/missing_cache_files.txt"

DEVICE = "cuda" if torch.cuda.is_available() else "cpu"
SFREQ  = 100
CROP_T = 1000
N_CHANS = 128  # Cz 제거 후
VAL_RELEASE = "R5"

# training
BATCH_SUBJ = 8          # subject batch size
K_WINDOWS  = 4          # windows per subject (hierarchical)
EPOCHS   = 20
LR       = 3e-4
WD       = 1e-2
SEED     = 42

# bands for fixed multiband front (Hz)
BANDS = [(0.5,4), (4,8), (8,13), (13,30), (30,45)]

warnings.filterwarnings("ignore")
mne.set_log_level("CRITICAL")
random.seed(SEED); np.random.seed(SEED); torch.manual_seed(SEED); torch.cuda.manual_seed_all(SEED)

# ============================================================
# 0. utils
# ============================================================
def subj_from_fname(path):
    base = os.path.basename(path)
    return base.split("_")[0]

def ensure_dir(p): os.makedirs(p, exist_ok=True)

# ============================================================
# 1. 파일 리스트 (.set 기준) - 네가 주신 로직 재사용
# ============================================================
def build_eeg_filelist(cache_dir, missing_txt, bids_root):
    cache_files = [f for f in os.listdir(cache_dir) if f.endswith(".npy")]
    found_from_cache, not_found = [], []

    for f in cache_files:
        subj = f.split("_")[0]
        task = f.split("_task-")[1].split("_")[0]
        run = None
        if "_run-" in f:
            run = f.split("_run-")[1].split("_")[0]
        matched = False
        for ds in os.listdir(bids_root):
            eeg_dir = os.path.join(bids_root, ds, subj, "eeg")
            if not os.path.exists(eeg_dir): continue
            for fname in os.listdir(eeg_dir):
                if not fname.endswith(".set"): continue
                if run is None:
                    if f"task-{task}" in fname:
                        found_from_cache.append(os.path.join(eeg_dir, fname)); matched=True; break
                else:
                    run_variants = [f"run-{run}", f"run-0{run}"]
                    if f"task-{task}" in fname and any(r in fname for r in run_variants):
                        found_from_cache.append(os.path.join(eeg_dir, fname)); matched=True; break
            if matched: break
        if not matched: not_found.append(f)

    found_from_missing = []
    if os.path.exists(missing_txt):
        with open(missing_txt, "r") as f:
            for line in f:
                base = line.strip()
                if not base: continue
                subj = base.split("_task-")[0]
                task = base.split("_task-")[1].split("_")[0]
                run = None
                if "_run-" in base:
                    run = base.split("_run-")[1].split("_")[0]
                for ds in os.listdir(bids_root):
                    eeg_dir = os.path.join(bids_root, ds, subj, "eeg")
                    if not os.path.exists(eeg_dir): continue
                    for fname in os.listdir(eeg_dir):
                        if not fname.endswith(".set"): continue
                        if run is None:
                            if f"task-{task}" in fname:
                                found_from_missing.append(os.path.join(eeg_dir, fname)); break
                        else:
                            run_variants = [f"run-{run}", f"run-0{run}"]
                            if f"task-{task}" in fname and any(r in fname for r in run_variants):
                                found_from_missing.append(os.path.join(eeg_dir, fname)); break

    all_files = sorted(list(set(found_from_cache + found_from_missing)))
    print(f"✅ Total EEG files: {len(all_files)} (cache={len(found_from_cache)}, missing={len(found_from_missing)})")
    if len(not_found) > 0:
        print(f"⚠️ not matched .set for {len(not_found)} cache keys (showing 10): {not_found[:10]}")
    return all_files

# ============================================================
# 2. participants + labels (+ subject-level stats)
# ============================================================
def load_participants(bids_root):
    dfs=[]
    for ds in sorted(os.listdir(bids_root)):
        pfile=os.path.join(bids_root, ds, "participants.tsv")
        if os.path.exists(pfile):
            df=pd.read_csv(pfile,sep="\t"); df["release"]=ds; dfs.append(df)
    meta=pd.concat(dfs,ignore_index=True)
    meta.columns=[c.lower() for c in meta.columns]
    meta=meta.drop_duplicates(subset=["participant_id"]).set_index("participant_id")

    # unify targets
    if "p_factor" not in meta.columns:
        alt=[c for c in meta.columns if "p" in c and "factor" in c]
        if alt: meta=meta.rename(columns={alt[0]:"p_factor"})
    for c in ["internalizing","externalizing"]:
        if c not in meta.columns: meta[c]=np.nan

    # meta features
    meta["sex"]=meta.get("sex", pd.Series(index=meta.index)).map({"M":1,"F":0,"m":1,"f":0}).fillna(0.5)
    for c in ["age","ehq_total"]:
        if c not in meta.columns: meta[c]=np.nan
        meta[c]=meta[c].fillna(meta[c].mean())
        m,s=meta[c].mean(), meta[c].std()+1e-8
        meta[c]=(meta[c]-m)/s
    return meta

# quick subject stats from cached npy (mean/std per channel) for subject-aware norm
def build_subject_stats(file_list):
    stats = {}
    for p in tqdm(file_list, desc="Subject stats"):
        subj = subj_from_fname(p)
        try:
            npy = try_load_or_cache_npy(p, write_cache=False)
            x = npy if npy is not None else None
            if x is None:  # fallback .set
                raw = mne.io.read_raw_eeglab(p, preload=True, verbose=False)
                raw.filter(0.5,45., fir_design="firwin")
                raw.set_eeg_reference("average", projection=False)
                raw.resample(SFREQ)
                x = raw.get_data().astype(np.float32)
        except Exception:
            continue
        if x.shape[0] == 129: x = x[:-1,:]
        mu = x.mean(1); sd = x.std(1)+1e-6
        stats[subj] = (mu.astype(np.float32), sd.astype(np.float32))
    return stats

# ============================================================
# 3. .npy 캐시 (있으면 사용 / 없으면 생성)
# ============================================================
def npy_cache_path(set_path):
    ensure_dir(NPY_CACHE)
    base = os.path.basename(set_path).replace(".set",".npy")
    return os.path.join(NPY_CACHE, base)

def try_load_or_cache_npy(set_path, write_cache=True):
    npy_path = npy_cache_path(set_path)
    if os.path.exists(npy_path):
        return np.load(npy_path, mmap_mode="r")
    if not write_cache: return None
    try:
        raw = mne.io.read_raw_eeglab(set_path, preload=True, verbose=False)
        raw.filter(0.5,45., fir_design="firwin")
        raw.set_eeg_reference("average", projection=False)
        raw.resample(SFREQ)
        x = raw.get_data().astype(np.float32)
        np.save(npy_path, x)
        return x
    except Exception:
        return None

# ============================================================
# 4. Multi-band fixed FIR kernels (torch conv)
# ============================================================
def design_fir_bandpass(l_hz, h_hz, fs=SFREQ, kernel=129, device="cpu", dtype=torch.float32):
    assert kernel % 2 == 1, "kernel must be odd for symmetric sinc."
    n = torch.arange(-(kernel//2), kernel//2 + 1, device=device, dtype=dtype)          # 129개

    nyq = fs/2.0
    f1, f2 = torch.tensor(l_hz/nyq, dtype=dtype, device=device), torch.tensor(h_hz/nyq, dtype=dtype, device=device)
    h1 = 2*f2*torch.sinc(2*f2*n); h2 = 2*f1*torch.sinc(2*f1*n)
    win = torch.hamming_window(n.numel(), periodic=False, device=device, dtype=dtype)    # 129개
    return ((h1-h2)*win).view(1,1,-1)  # (out=1, in=1, K)

class MultiBandFront(nn.Module):
    """(B,C,T) -> (B, C*nbands, T) by fixed FIR bank"""
    def __init__(self, bands=BANDS, fs=SFREQ, kernel=129):
        super().__init__()
        self.kernels = nn.ParameterList()
        for l,h in bands:
            k = design_fir_bandpass(l,h,fs,kernel)
            p = nn.Parameter(k, requires_grad=False)
            self.kernels.append(p)
        self.nbands = len(bands)
        self.kernel = kernel
    def forward(self,x):  # x:(B,C,T)
        B,C,T = x.shape
        outs=[]
        x_dw = x.view(B*C,1,T)
        for k in self.kernels:
            y = F.conv1d(x, k, stride=1, padding=self.kernel_size // 2)
            outs.append(y)
        y = torch.cat(outs, dim=1)              # (B*C, nbands, T)
        y = y.view(B, C*self.nbands, T)
        return y

# ============================================================
# 5. Model: window encoder + subject Transformer + meta head
# ============================================================
class WindowBackbone(nn.Module):
    """CBraMod-like conv on multiband channels"""
    def __init__(self, in_ch, out_dim=256):
        super().__init__()
        self.conv = nn.Sequential(
            nn.Conv1d(in_ch, 128, 7, padding=3), nn.ReLU(),
            nn.Conv1d(128, 256, 5, padding=2), nn.ReLU(),
            nn.AdaptiveAvgPool1d(1),
        )
        self.fc = nn.Linear(256, out_dim)
    def forward(self, x):  # x:(B, Cmb, T) -> (B, D)
        h = self.conv(x).squeeze(-1)
        return self.fc(h)

class SubjectTransformer(nn.Module):
    """aggregate K window features (B,K,D)->(B,D)"""
    def __init__(self, d=256, heads=4, layers=2, dropout=0.1):
        super().__init__()
        enc = nn.TransformerEncoderLayer(d_model=d, nhead=heads, batch_first=True, dropout=dropout)
        self.enc = nn.TransformerEncoder(enc, num_layers=layers)
        self.norm = nn.LayerNorm(d)
    def forward(self, f):       # f:(B,K,D)
        z = self.enc(f)
        z = self.norm(z)
        return z.mean(1)        # (B,D)

class MultiTaskHead(nn.Module):
    def __init__(self, in_dim, meta_dim=3, tasks=3):
        super().__init__()
        self.meta = nn.Sequential(nn.Linear(meta_dim,32), nn.ReLU())
        self.head = nn.Sequential(nn.Linear(in_dim+32,128), nn.ReLU(), nn.Linear(128,tasks))
        self.tasks = tasks
    def forward(self, sfeat, meta):
        m = self.meta(meta)
        out = self.head(torch.cat([sfeat, m], 1))
        return out  # (B, tasks) -> [p, internalizing, externalizing]

# ============================================================
# 6. Dataset: subject batch with K windows (hierarchical)
# ============================================================
class SubjectDataset(Dataset):
    def __init__(self, files, meta, stats, train=True, k=K_WINDOWS):
        self.files = files
        self.meta = meta
        self.stats = stats    # dict subj -> (mu_ch, sd_ch)
        self.train = train
        self.k = k
        # targets available subjects
        self.targets = meta[["p_factor","internalizing","externalizing"]].copy()
        self.targets = self.targets.dropna(subset=["p_factor"])
        self.subjs = sorted(list(set([subj_from_fname(p) for p in self.files]) & set(self.targets.index)))

    def __len__(self): return len(self.subjs)

    def _read_eeg(self, path):
        # try cache
        x = try_load_or_cache_npy(path)
        if x is None:
            raw=mne.io.read_raw_eeglab(path,preload=True,verbose=False)
            raw.filter(0.5,45.,fir_design="firwin")
            raw.set_eeg_reference("average",projection=False)
            raw.resample(SFREQ)
            x=raw.get_data().astype(np.float32)
        if x.shape[0]==129: x=x[:-1,:]
        return x

    def _pick_crop(self, x):
        T=x.shape[1]
        if T<CROP_T:
            pad=np.zeros((x.shape[0],CROP_T),dtype=np.float32); pad[:,:T]=x; x=pad
        else:
            st=np.random.randint(0,T-CROP_T+1) if self.train else (T-CROP_T)//2
            x=x[:,st:st+CROP_T]
        return x

    def __getitem__(self, i):
        subj=self.subjs[i]
        # 파일들 중 해당 subject 모든 파일
        cand=[p for p in self.files if subj_from_fname(p)==subj]
        sel=random.choices(cand, k=self.k)

        Xs=[]
        for p in sel:
            x=self._read_eeg(p)
            # subject-aware norm
            if subj in self.stats:
                mu,sd=self.stats[subj]
                x=(x-mu[:,None])/(sd[:,None])
            else:
                x=(x-x.mean(1,keepdims=True))/(x.std(1,keepdims=True)+1e-6)
            x=self._pick_crop(x)
            Xs.append(x)
        X=np.stack(Xs,0)  # (K, C, T)

        # meta & targets
        r=self.meta.loc[subj]
        meta_vec=np.array([r["age"], r["sex"], r["ehq_total"]],dtype=np.float32)
        y=np.array([
            r.get("p_factor", np.nan),
            r.get("internalizing", np.nan),
            r.get("externalizing", np.nan)
        ], dtype=np.float32)

        return torch.tensor(X), torch.tensor(meta_vec), torch.tensor(y), subj

# ============================================================
# 7. Augment & Loss
# ============================================================
def light_augment(x):  # x:(B,K,C,T)
    # gaussian jitter
    x = x + 0.01*torch.randn_like(x)
    # time mask (SpecAugment-style on T)
    B,K,C,T = x.shape
    if T>50:
        w = int(0.05*T)
        t0 = torch.randint(0, T-w+1, (B*K,))
        for b in range(B*K):
            x.view(B*K,C,T)[b,:,t0[b]:t0[b]+w] = 0
    return x

def mixup(x, y, alpha=0.2):
    if alpha<=0: return x, y
    lam = np.random.beta(alpha, alpha)
    idx = torch.randperm(x.size(0), device=x.device)
    x2, y2 = x[idx], y[idx]
    return lam*x + (1-lam)*x2, lam*y + (1-lam)*y2

def corr_loss(y_true, y_pred):
    vx = y_pred - y_pred.mean()
    vy = y_true - y_true.mean()
    num = (vx*vy).sum()
    den = torch.sqrt((vx**2).sum())*torch.sqrt((vy**2).sum()) + 1e-8
    return 1 - num/den

# ============================================================
# 8. Train
# ============================================================
class ModelAll(nn.Module):
    def __init__(self, n_ch=N_CHANS, bands=BANDS, sfreq=SFREQ, crop=CROP_T,
                 win_out=256, heads=4, layers=2):
        super().__init__()
        self.front = MultiBandFront(bands=bands, fs=sfreq, kernel=129)
        self.backbone = WindowBackbone(in_ch=n_ch*len(bands), out_dim=win_out)
        self.tr = SubjectTransformer(d=win_out, heads=heads, layers=layers)
        self.head = MultiTaskHead(in_dim=win_out, meta_dim=3, tasks=3)

    def forward(self, X, meta):  # X:(B,K,C,T)
        B,K,C,T = X.shape
        X = X.view(B*K, C, T)
        mb = self.front(X)                          # (B*K, C*nb, T)
        f  = self.backbone(mb)                      # (B*K, D)
        f  = f.view(B, K, -1)                       # (B,K,D)
        s  = self.tr(f)                             # (B,D)
        out= self.head(s, meta)                     # (B,3)
        return out, s

def train_one_epoch(model, loader, opt, scaler, mixup_alpha=0.2):
    model.train()
    tot=0; n=0
    for X, meta, y, _ in tqdm(loader, desc="Train", leave=False):
        X,meta,y = X.float().to(DEVICE), meta.float().to(DEVICE), y.float().to(DEVICE)
        X = light_augment(X)
        opt.zero_grad(set_to_none=True)
        with torch.cuda.amp.autocast(True):
            # mixup on subject-level targets only for available labels (NaN-safe later)
            y_mix = y.clone()
            Xm, ym = mixup(X, y_mix, alpha=mixup_alpha)
            pred, _ = model(Xm, meta)
            # mask (some targets may be NaN)
            mask = ~torch.isnan(ym)
            mse = ((pred - ym)[mask]**2).mean()
            corr = corr_loss(ym[:,0][~torch.isnan(ym[:,0])], pred[:,0][~torch.isnan(pred[:,0])])
            loss = mse + 0.2*corr
        scaler.scale(loss).backward()
        scaler.step(opt); scaler.update()
        tot += float(loss.detach()); n += 1
    return tot/max(n,1)

@torch.no_grad()
def evaluate(model, loader):
    model.eval()
    yh=[]; yt=[]
    for X, meta, y, _ in tqdm(loader, desc="Val", leave=False):
        X,meta = X.float().to(DEVICE), meta.float().to(DEVICE)
        pred,_ = model(X, meta)
        yh.append(pred.cpu().numpy()); yt.append(y.numpy())
    yh = np.concatenate(yh,0); yt = np.concatenate(yt,0)
    # metrics for p_factor (col 0)
    mask = ~np.isnan(yt[:,0])
    pred_p = yh[:,0][mask]; true_p = yt[:,0][mask]
    mae = np.mean(np.abs(pred_p-true_p))
    mse = np.mean((pred_p-true_p)**2)
    r2  = 1 - np.sum((pred_p-true_p)**2)/(np.sum((true_p-true_p.mean())**2)+1e-8)
    return mae, mse, r2

# ============================================================
# 9. Run
# ============================================================
def main():
    files = build_eeg_filelist(CACHE_DIR, MISSING_TXT, BIDS_ROOT)
    meta  = load_participants(BIDS_ROOT)
    # split by release name in path
    train_files=[f for f in files if VAL_RELEASE not in f]
    val_files  =[f for f in files if VAL_RELEASE in f]

    # subject-level mean/std for normalization (from cache if possible)
    stats = build_subject_stats(train_files)

    train_ds = SubjectDataset(train_files, meta, stats, train=True,  k=K_WINDOWS)
    val_ds   = SubjectDataset(val_files,   meta, stats, train=False, k=K_WINDOWS)
    train_loader = DataLoader(train_ds, batch_size=BATCH_SUBJ, shuffle=True,
                              num_workers=4, pin_memory=True, prefetch_factor=4, drop_last=True)
    val_loader   = DataLoader(val_ds,   batch_size=BATCH_SUBJ, shuffle=False,
                              num_workers=2, pin_memory=True)

    model = ModelAll(n_ch=N_CHANS, bands=BANDS, sfreq=SFREQ, crop=CROP_T,
                     win_out=256, heads=4, layers=2).to(DEVICE)
    opt = torch.optim.AdamW(model.parameters(), lr=LR, weight_decay=WD)
    scaler = torch.cuda.amp.GradScaler()

    best = {"mse": 1e9, "epoch": -1}
    for ep in range(1, EPOCHS+1):
        t0=time.time()
        tr = train_one_epoch(model, train_loader, opt, scaler, mixup_alpha=0.2)
        mae, mse, r2 = evaluate(model, val_loader)
        print(f"[Ep{ep:02d}] TrainLoss={tr:.4f} | Val MSE={mse:.4f} MAE={mae:.4f} R2={r2:.3f} time={time.time()-t0:.1f}s")
        if mse < best["mse"]:
            best = {"mse": float(mse), "epoch": ep}
            torch.save(model.state_dict(), "best_c2_hier_mb.pt")
            with open("best_c2_metrics.json","w") as f:
                json.dump({"MAE": float(mae), "MSE": float(mse), "R2": float(r2)}, f, indent=2)
    print("✅ Best:", best)

if __name__=="__main__":
    main()


✅ Total EEG files: 340 (cache=318, missing=23)


Subject stats: 100%|██████████| 336/336 [2:00:12<00:00, 21.46s/it]  


AttributeError: 'MultiBandFront' object has no attribute 'kernel_size'

## 2D CNN으로

In [37]:
# ============================================================
# CHALLENGE 2: p-factor regression using 2D CNN model
# ============================================================

import os
import torch
import torch.nn as nn
import torch.nn.functional as F
from torch.utils.data import Dataset, DataLoader
import numpy as np
import pandas as pd
from tqdm import tqdm
from sklearn.metrics import mean_absolute_error, mean_squared_error, r2_score

# ============================================================
# Configuration
# ============================================================
CACHE_DIR = "/data5/open_data/HBN/cache_eeg_windows"
BIDS_ROOT = "/data5/open_data/HBN/EEG_BIDS"
DATASETS  = [f"ds0055{i:02d}" for i in range(5, 17)]
DEVICE    = "cuda" if torch.cuda.is_available() else "cpu"

N_CHANS = 129     # 캐시 shape[1]
WIN_SIZE = 500   # 캐시 shape[2]
SFREQ = 100       # Hz

print(f"Device: {DEVICE}")

# ============================================================
# participants.tsv 병합 및 p_factor 로드
# ============================================================
def load_all_participants(bids_root=BIDS_ROOT, datasets=DATASETS):
    dfs = []
    for ds in datasets:
        pfile = os.path.join(bids_root, ds, "participants.tsv")
        if os.path.exists(pfile):
            df = pd.read_csv(pfile, sep="\t")
            df["release"] = ds
            dfs.append(df)
    df_all = pd.concat(dfs, ignore_index=True)
    df_all = df_all.drop_duplicates(subset=["participant_id"])
    df_all = df_all.set_index("participant_id")
    return df_all

meta_df   = load_all_participants()
labels_df = meta_df[["p_factor"]].dropna()

# ============================================================
# 캐시 파일 목록 불러오기
# ============================================================
all_cache_files = [f for f in os.listdir(CACHE_DIR) if f.endswith(".npy")]
train_files = [f for f in all_cache_files if "_train_" in f]
val_files   = [f for f in all_cache_files if "_val_" in f]

print(f"Train cache files: {len(train_files)}")
print(f"Val cache files:   {len(val_files)}")

# ============================================================
# Dataset 정의 (2D CNN 전용)
# ============================================================
class EEGCropDataset(Dataset):
    def __init__(self, cache_dir, cache_files, labels_df,
                 crop_size=500, n_crops=3, augment=True):
        self.cache_dir = cache_dir
        self.cache_files = cache_files
        self.labels_df = labels_df
        self.crop_size = crop_size
        self.n_crops = n_crops
        self.augment = augment
        self.samples = []

        for fname in tqdm(cache_files, desc="Indexing EEG crops"):
            subj = fname.split("_")[0]
            if subj not in labels_df.index:
                continue
            y = float(labels_df.loc[subj, "p_factor"])
            path = os.path.join(cache_dir, fname)
            # 파일 열지 않고 shape만 저장
            arr = np.load(path, mmap_mode='r')
            n_windows = arr.shape[0]
            arr = None
            for w in range(n_windows):
                self.samples.append((fname, w, y))

    def __len__(self):
        return len(self.samples)

    def __getitem__(self, idx):
        fname, w_idx, y = self.samples[idx]
        path = os.path.join(self.cache_dir, fname)
        f = np.load(path, mmap_mode='r')        # 메모리 매핑
        Xw = f[w_idx]                           # 필요한 window만 로드
        f = None

        max_start = Xw.shape[1] - self.crop_size
        if self.augment:
            start = np.random.randint(0, max_start)
        else:
            start = max_start // 2
        crop = Xw[:, start:start+self.crop_size]

        X_tensor = torch.tensor(crop, dtype=torch.float32).unsqueeze(0)
        y_tensor = torch.tensor(y, dtype=torch.float32)
        return X_tensor, y_tensor


# ============================================================
# DataLoader
# ============================================================
train_ds = EEGCropDataset(CACHE_DIR, train_files, labels_df,
                          crop_size=500, n_crops=3, augment=True)
val_ds   = EEGCropDataset(CACHE_DIR, val_files, labels_df,
                          crop_size=500, n_crops=1, augment=False)

train_loader = DataLoader(train_ds, batch_size=16, shuffle=True, num_workers=0, pin_memory=True)
val_loader   = DataLoader(val_ds, batch_size=16, shuffle=False, num_workers=0, pin_memory=True)

# ============================================================
# 2D CNN 모델 정의
# ============================================================
class EEG_CNN2D(nn.Module):
    """
    EEG 2D CNN baseline for p-factor regression
    입력: (B, 1, 129, 1000)
    출력: (B, 1)
    """
    def __init__(self, n_chans=129, n_times=500, n_outputs=1, dropout=0.3):
        super().__init__()

        self.conv1 = nn.Conv2d(1, 16, kernel_size=(3, 25), padding=(1, 12))
        self.bn1   = nn.BatchNorm2d(16)
        self.pool1 = nn.MaxPool2d((2, 4))  # (C,T) -> (C/2,T/4)

        self.conv2 = nn.Conv2d(16, 32, kernel_size=(3, 15), padding=(1, 7))
        self.bn2   = nn.BatchNorm2d(32)
        self.pool2 = nn.MaxPool2d((2, 4))  # (C/4,T/16)

        self.conv3 = nn.Conv2d(32, 64, kernel_size=(3, 7), padding=(1, 3))
        self.bn3   = nn.BatchNorm2d(64)
        self.pool3 = nn.AdaptiveAvgPool2d((1, 1))  # Global avg pooling

        self.drop  = nn.Dropout(dropout)
        self.fc    = nn.Linear(64, n_outputs)

    def forward(self, x):
        # (B, 1, chans, times)
        x = F.elu(self.bn1(self.conv1(x)))
        x = self.pool1(x)
        x = F.elu(self.bn2(self.conv2(x)))
        x = self.pool2(x)
        x = F.elu(self.bn3(self.conv3(x)))
        x = self.pool3(x)
        x = self.drop(x)
        x = torch.flatten(x, 1)
        out = self.fc(x)
        return out.squeeze(-1)

# ============================================================
# 학습 함수
# ============================================================
def train_model(model, train_loader, val_loader, n_epochs=30, lr=1e-4):
    model = model.to(DEVICE)
    opt = torch.optim.AdamW(model.parameters(), lr=lr, weight_decay=1e-3)
    best_mae, best_mse, best_r2 = float("inf"), float("inf"), -1.0

    for ep in range(1, n_epochs + 1):
        # ---------------------- Training ----------------------
        model.train()
        train_mae, train_mse = 0.0, 0.0
        for X, y in tqdm(train_loader, desc=f"[Epoch {ep}/{n_epochs}] (train)", leave=False):
            X, y = X.to(DEVICE), y.to(DEVICE)
            pred = model(X)
            pred = pred.view(-1)
            y = y.view(-1)
            loss_mae = F.l1_loss(pred, y)
            loss_mse = F.mse_loss(pred, y)
            loss = 0.5 * (loss_mae + loss_mse)
            opt.zero_grad(); loss.backward(); opt.step()
            train_mae += loss_mae.item(); train_mse += loss_mse.item()
        avg_mae = train_mae / len(train_loader)
        avg_mse = train_mse / len(train_loader)

        # ---------------------- Validation ----------------------
        model.eval(); preds, trues = [], []
        with torch.no_grad():
            for X, y in tqdm(val_loader, desc=f"[Epoch {ep}/{n_epochs}] (val)", leave=False):
                X, y = X.to(DEVICE), y.to(DEVICE)
                p = model(X).view(-1)
                preds.extend(p.cpu().numpy().tolist())
                trues.extend(y.cpu().numpy().tolist())
        preds, trues = np.array(preds), np.array(trues)
        val_mae = mean_absolute_error(trues, preds)
        val_mse = mean_squared_error(trues, preds)
        val_r2  = r2_score(trues, preds)

        print(f"Epoch {ep:02d} | "
              f"Train MAE={avg_mae:.4f}, MSE={avg_mse:.4f} | "
              f"Val MAE={val_mae:.4f}, MSE={val_mse:.4f}, R²={val_r2:.3f}")

        if val_mae < best_mae:
            best_mae, best_mse, best_r2 = val_mae, val_mse, val_r2
            torch.save(model.state_dict(), "best_EEG_CNN2D_aug.pth")

    print(f"\n✅ Best Val | MAE={best_mae:.4f}, MSE={best_mse:.4f}, R²={best_r2:.3f}")
    return best_mae, best_mse, best_r2

# ============================================================
# 실행
# ============================================================
model = EEG_CNN2D(n_chans=N_CHANS, n_times=WIN_SIZE)
mae, mse, r2 = train_model(model, train_loader, val_loader, n_epochs=30)
print(f"\n=== Final 2D CNN Results ===\nMAE={mae:.4f} | MSE={mse:.4f} | R²={r2:.4f}")


Device: cuda
Train cache files: 309
Val cache files:   8


Indexing EEG crops: 100%|██████████| 8/8 [00:00<00:00, 259.55it/s]


Epoch 01 | Train MAE=0.7200, MSE=0.8161 | Val MAE=0.8507, MSE=1.0585, R²=-0.001


Epoch 02 | Train MAE=0.7176, MSE=0.8085 | Val MAE=0.8761, MSE=1.0703, R²=-0.012


Epoch 03 | Train MAE=0.7169, MSE=0.8070 | Val MAE=0.8470, MSE=1.0466, R²=0.011


...

Epoch 26 | Train MAE=0.6992, MSE=0.7804 | Val MAE=0.8044, MSE=1.0307, R²=0.026


Epoch 27 | Train MAE=0.6989, MSE=0.7798 | Val MAE=0.8790, MSE=1.0555, R²=0.002


Epoch 28 | Train MAE=0.6983, MSE=0.7793 | Val MAE=0.8462, MSE=1.0383, R²=0.019


Epoch 29 | Train MAE=0.6982, MSE=0.7792 | Val MAE=0.8550, MSE=1.0499, R²=0.008


Epoch 30 | Train MAE=0.6980, MSE=0.7786 | Val MAE=0.8448, MSE=1.0196, R²=0.036

✅ Best Val | MAE=0.8044, MSE=1.0307, R²=0.026

=== Final 2D CNN Results ===
MAE=0.8044 | MSE=1.0307 | R²=0.0257


In [3]:
import numpy as np
locs = np.stack([np.array(ch['loc']) for ch in chs_info])
print("Has NaN in loc:", np.isnan(locs).any())


Has NaN in loc: True


In [13]:
# ============================================================
# CHALLENGE 2: p-factor regression with LaBraM (Braindecode)
# - StratifiedGroupKFold CV
# - biosemi128 (128 ch) 기본, 129ch 파일은 앞 128ch로 슬라이스
# - z-score + clipping + micro-noise (안정화)
# ============================================================

import os
import numpy as np
import pandas as pd
import torch
import torch.nn as nn
import torch.nn.functional as F
from torch.utils.data import Dataset, DataLoader
from tqdm import tqdm
from sklearn.metrics import mean_absolute_error, mean_squared_error, r2_score
from sklearn.model_selection import StratifiedGroupKFold

# ⬇️ 핵심: LaBraM
from braindecode.models import Labram  # (batch, n_chans, n_times) when neural_tokenizer=True

# -------------------------
# Configuration
# -------------------------
CACHE_DIR = "/data5/open_data/HBN/cache_eeg_windows"
BIDS_ROOT = "/data5/open_data/HBN/EEG_BIDS"
DATASETS  = [f"ds0055{i:02d}" for i in range(5, 17)]
DEVICE    = "cuda" if torch.cuda.is_available() else "cpu"

N_CHANS   = 128      # 기본: biosemi128 (Cz 제외). 129ch 쓰려면 129로 변경하고 아래 슬라이스 주석 처리
WIN_SIZE  = 1000     # time points
SFREQ     = 100
BATCH_SIZE = 4
EPOCHS     = 25
LR         = 5e-5
N_FOLDS    = 5

DEBUG = True
torch.backends.cudnn.benchmark = False
torch.backends.cudnn.deterministic = True
torch.set_float32_matmul_precision("high")

print(f"Device: {DEVICE}")

# ------------------------------------------------------------
# 1) participants.tsv 병합 및 p_factor 로드
# ------------------------------------------------------------
def load_all_participants(bids_root, datasets):
    dfs = []
    for ds in datasets:
        pfile = os.path.join(bids_root, ds, "participants.tsv")
        if os.path.exists(pfile):
            df = pd.read_csv(pfile, sep="\t")
            df["release"] = ds
            dfs.append(df)
    df_all = pd.concat(dfs, ignore_index=True)
    df_all = df_all.drop_duplicates(subset=["participant_id"])
    df_all = df_all.set_index("participant_id")
    return df_all

meta_df   = load_all_participants(BIDS_ROOT, DATASETS)
labels_df = meta_df[["p_factor"]].dropna().copy()
labels_df = labels_df[labels_df["p_factor"].apply(np.isfinite)]

# ------------------------------------------------------------
# 2) 캐시 파일 목록
# ------------------------------------------------------------
all_files = [f for f in os.listdir(CACHE_DIR) if f.endswith(".npy")]
train_pool_files = [f for f in all_files if "_train_" in f]
ext_val_files    = [f for f in all_files if "_val_"   in f]  # (선택) release5 hold-out

def subj_from_fname(fname): return fname.split("_")[0]

# ------------------------------------------------------------
# 3) StratifiedGroupKFold 준비 (p-factor를 분위 bin으로)
# ------------------------------------------------------------
rows = []
for f in train_pool_files:
    subj = subj_from_fname(f)
    if subj in labels_df.index:
        rows.append({"fname": f, "subj": subj, "p": float(labels_df.loc[subj, "p_factor"])})
df = pd.DataFrame(rows)
df["p_bin"] = pd.qcut(df["p"], q=5, labels=False, duplicates="drop")

cv = StratifiedGroupKFold(n_splits=N_FOLDS, shuffle=True, random_state=42)
folds = list(cv.split(df, y=df["p_bin"], groups=df["subj"]))
print(df["p_bin"].value_counts().sort_index())

# ------------------------------------------------------------
# 4) Dataset 정의 (z-score + clip + micro-noise)
# ------------------------------------------------------------
class EEGCachedDataset(Dataset):
    def __init__(self, cache_dir, rows, labels_df, n_segments_per_item=4, train=True):
        self.cache_dir = cache_dir
        self.rows = rows.reset_index(drop=True)
        self.labels_df = labels_df
        self.n_segments_per_item = n_segments_per_item
        self.train = train

    def __len__(self): return len(self.rows)

    def __getitem__(self, idx):
        r = self.rows.iloc[idx]
        path = os.path.join(self.cache_dir, r.fname)

        # X_all: (segments, C, T) 또는 (segments, T, C)
        X_all = np.load(path, allow_pickle=True)
        X_all = np.nan_to_num(X_all, nan=0.0, posinf=0.0, neginf=0.0)

        # 여러 segment 중 일부만 사용 → 평균 pooling
        S = X_all.shape[0]
        k = min(self.n_segments_per_item, S)
        sel = np.random.choice(S, size=k, replace=(S < k)) if self.train else np.arange(S)
        X = X_all[sel].mean(axis=0)  # (C, T) 또는 (T, C)

        # 축 정렬: (C, T)
        if X.shape[0] != N_CHANS and X.shape[1] == N_CHANS:
            X = X.T

        # 채널 수 강제 일치 (캐시에 129ch가 섞여있을 수 있음)
        if X.shape[0] > N_CHANS:
            X = X[:N_CHANS, :]
        elif X.shape[0] < N_CHANS:
            # 부족하면 zero-pad (드물겠지만 안전)
            pad_c = N_CHANS - X.shape[0]
            X = np.pad(X, ((0, pad_c), (0, 0)), mode="constant")

        # 안정적 정규화: z-score + clip
        mean, std = np.mean(X), np.std(X)
        std = 1e-6 if (not np.isfinite(std) or std < 1e-6) else std
        X = (X - mean) / std
        X = np.clip(X, -3.0, 3.0)

        X = torch.tensor(X, dtype=torch.float32).contiguous()
        if self.train:
            X = X + 1e-5 * torch.randn_like(X)  # micro-noise (LayerNorm 안정화에 도움)

        y = torch.tensor(float(self.labels_df.loc[r.subj, "p_factor"]), dtype=torch.float32)
        return X, y

# ------------------------------------------------------------
# 5) LaBraM 회귀 모델 래퍼
#    (LaBraM은 n_outputs=1로 바로 회귀 head를 내부에 포함)
# ------------------------------------------------------------
class LabramRegressor(nn.Module):
    def __init__(self, n_chans=N_CHANS, n_times=WIN_SIZE, sfreq=SFREQ):
        super().__init__()
        self.backbone = Labram(
            n_chans=n_chans,
            n_times=n_times,
            sfreq=sfreq,
            n_outputs=1,             # ← 회귀
            neural_tokenizer=True,   # ← (B, C, T) 입력 허용
            use_mean_pooling=True,   # 최종 시퀀스 풀링 사용
            drop_prob=0.0,
            attn_drop_prob=0.0,
            drop_path_prob=0.0,
            emb_size=200,            # 기본값 (필요시 튜닝)
            patch_size=200,          # 1000 시계열이면 patch 200 → token 5개
            n_layers=8,              # 기본 12 → 경량화 (메모리/안정성)
            att_num_heads=8,
        )

        # 안정화를 위해 LayerNorm eps 확대
        for m in self.backbone.modules():
            if isinstance(m, nn.LayerNorm):
                m.eps = 1e-3

    def forward(self, x):
        # x: (B, C, T)
        out = self.backbone(x)      # (B, 1)
        return out.squeeze(-1)      # (B,)

# ------------------------------------------------------------
# 6) 1-fold 학습/검증
# ------------------------------------------------------------
def train_one_fold(train_rows, valid_rows, fold_id):
    train_ds = EEGCachedDataset(CACHE_DIR, train_rows, labels_df, train=True)
    valid_ds = EEGCachedDataset(CACHE_DIR, valid_rows, labels_df, train=False)

    train_loader = DataLoader(train_ds, batch_size=BATCH_SIZE, shuffle=True,  num_workers=4, pin_memory=True)
    valid_loader = DataLoader(valid_ds, batch_size=BATCH_SIZE, shuffle=False, num_workers=4, pin_memory=True)

    model = LabramRegressor().to(DEVICE)
    opt = torch.optim.AdamW(model.parameters(), lr=LR, weight_decay=1e-3)

    best = {"mae": np.inf, "mse": np.inf, "r2": -1}
    for ep in range(1, EPOCHS + 1):
        # ------- Train -------
        model.train()
        train_loss = 0.0
        for X, y in tqdm(train_loader, desc=f"[Fold {fold_id}] Epoch {ep} (train)", leave=False):
            X, y = X.to(DEVICE), y.to(DEVICE)
            pred = model(X)

            if torch.isnan(pred).any() or torch.isinf(pred).any():
                if DEBUG: print("[Train] NaN/Inf in pred → skip")
                continue

            loss = 0.5 * F.l1_loss(pred, y) + 0.5 * F.mse_loss(pred, y)
            if torch.isnan(loss) or torch.isinf(loss):
                if DEBUG: print("[Train] NaN/Inf loss → skip")
                continue

            opt.zero_grad()
            loss.backward()
            torch.nn.utils.clip_grad_norm_(model.parameters(), max_norm=0.5)
            opt.step()
            train_loss += loss.item()

        # ------- Val -------
        model.eval(); preds, trues = [], []
        with torch.no_grad():
            for X, y in tqdm(valid_loader, desc=f"[Fold {fold_id}] Epoch {ep} (val)", leave=False):
                X, y = X.to(DEVICE), y.to(DEVICE)
                p = model(X)
                preds.extend(p.detach().cpu().numpy())
                trues.extend(y.cpu().numpy())

        preds = np.array(preds, dtype=np.float32)
        trues = np.array(trues, dtype=np.float32)
        mask = np.isfinite(preds) & np.isfinite(trues)
        preds, trues = preds[mask], trues[mask]

        if len(trues) == 0:
            val_mae = val_mse = val_r2 = np.nan
        else:
            val_mae = mean_absolute_error(trues, preds)
            val_mse = mean_squared_error(trues, preds)
            val_r2  = r2_score(trues, preds)

        print(f"[Fold {fold_id}] Ep{ep:02d} | train_loss={train_loss/max(1,len(train_loader)):.4f} "
              f"| MAE={val_mae:.4f} | MSE={val_mse:.4f} | R²={val_r2:.3f}")

        if np.isfinite(val_mae) and (val_mae < best["mae"]):
            best = {"mae": val_mae, "mse": val_mse, "r2": val_r2}
            torch.save(model.state_dict(), f"labram_fold{fold_id}.pth")

    return best

# ------------------------------------------------------------
# 7) Cross-validation 실행
# ------------------------------------------------------------
cv_results = []
for fold_id, (tr_idx, va_idx) in enumerate(folds, 1):
    train_rows = df.iloc[tr_idx]
    valid_rows = df.iloc[va_idx]
    best = train_one_fold(train_rows, valid_rows, fold_id)
    cv_results.append(best)

mean_mae = np.nanmean([r["mae"] for r in cv_results])
mean_mse = np.nanmean([r["mse"] for r in cv_results])
mean_r2  = np.nanmean([r["r2"] for r in cv_results])

print("\n=== StratifiedGroupKFold CV Results ===")
print(f"MAE={mean_mae:.4f} | MSE={mean_mse:.4f} | R²={mean_r2:.4f}")

# ------------------------------------------------------------
# 8) (선택) Release 5 Hold-out 검증
# ------------------------------------------------------------
if len(ext_val_files) > 0:
    ext_rows = []
    for f in ext_val_files:
        subj = subj_from_fname(f)
        if subj in labels_df.index:
            ext_rows.append({"fname": f, "subj": subj})
    ext_df = pd.DataFrame(ext_rows)

    ext_ds = EEGCachedDataset(CACHE_DIR, ext_df, labels_df, train=False)
    ext_loader = DataLoader(ext_ds, batch_size=BATCH_SIZE, shuffle=False, num_workers=4, pin_memory=True)

    model = LabramRegressor().to(DEVICE)
    state = torch.load("labram_fold1.pth", map_location=DEVICE)
    model.load_state_dict(state)
    model.eval()

    preds, trues = [], []
    with torch.no_grad():
        for X, y in tqdm(ext_loader, desc="External Val (Release5)"):
            X, y = X.to(DEVICE), y.to(DEVICE)
            p = model(X)
            preds.extend(p.cpu().numpy())
            trues.extend(y.cpu().numpy())

    preds, trues = np.array(preds, dtype=np.float32), np.array(trues, dtype=np.float32)
    mask = np.isfinite(preds) & np.isfinite(trues)
    preds, trues = preds[mask], trues[mask]

    val_mae = mean_absolute_error(trues, preds)
    val_mse = mean_squared_error(trues, preds)
    val_r2  = r2_score(trues, preds)
    print(f"\n[Release5 Hold-out] MAE={val_mae:.4f} | MSE={val_mse:.4f} | R²={val_r2:.4f}")


Device: cuda
p_bin
0    62
1    62
2    61
3    62
4    62
Name: count, dtype: int64


[Fold 1] Ep01 | train_loss=0.7395 | MAE=0.7656 | MSE=0.9835 | R²=-0.027


[Fold 1] Ep02 | train_loss=0.7353 | MAE=0.7692 | MSE=0.9943 | R²=-0.038


[Fold 1] Ep03 | train_loss=0.7379 | MAE=0.7645 | MSE=0.9769 | R²=-0.020


[Fold 1] Ep04 | train_loss=0.7440 | MAE=0.7632 | MSE=0.9713 | R²=-0.014


...

[Fold 5] Ep23 | train_loss=0.7668 | MAE=0.7112 | MSE=0.8083 | R²=-0.048


[Fold 5] Ep24 | train_loss=0.7613 | MAE=0.7118 | MSE=0.8074 | R²=-0.047


[Fold 5] Ep25 | train_loss=0.7683 | MAE=0.7124 | MSE=0.8046 | R²=-0.044

=== StratifiedGroupKFold CV Results ===
MAE=0.7081 | MSE=0.8109 | R²=-0.0194


External Val (Release5): 100%|██████████| 2/2 [00:05<00:00,  2.58s/it]


[Release5 Hold-out] MAE=0.8691 | MSE=1.2294 | R²=-0.0168


In [22]:
# ============================================================
# CHALLENGE 2: p-factor regression with pretrained CBraMod
# - Uses cached EEG .npy (segments, C, T)  e.g. (Sg, 129, 1000)
# - Converts (C, T) -> (C, S, P) patches to feed CBraMod
# - Loads pretrained weights; adds regression head
# - StratifiedGroupKFold by subject with p-factor quantile bins
# ============================================================

import os
import numpy as np
import pandas as pd
from tqdm import tqdm

import torch
import torch.nn as nn
import torch.nn.functional as F
from torch.utils.data import Dataset, DataLoader

from sklearn.metrics import mean_absolute_error, mean_squared_error, r2_score
from sklearn.model_selection import StratifiedGroupKFold

# --- CBraMod (from repo)
#   model class path: models.cbramod.CBraMod (README Quick Start)  [b, c, s, p]  → output features
#   We'll set proj_out = nn.Identity() and add our own regression head.

import sys
sys.path.append("/home/RA/EEG_Challenge/Challenge2/CBraMod/")  # clone한 경로로 수정

from models.cbramod import CBraMod  # repo 구조 그대로 사용  (README Quick Start)  # :contentReference[oaicite:4]{index=4}

# -------------------------
# Configuration
# -------------------------
CACHE_DIR       = "/data5/open_data/HBN/cache_eeg_windows"
BIDS_ROOT       = "/data5/open_data/HBN/EEG_BIDS"
DATASETS        = [f"ds0055{i:02d}" for i in range(5, 17)]
DEVICE          = "cuda" if torch.cuda.is_available() else "cpu"

# Data specs
N_CHANS         = 129      # BioSemi128 + Cz
T_TOTAL         = 1000     # time points per window
SFREQ           = 100

# Patch settings for CBraMod
PATCH_SIZE      = 200      # P (points per patch)
PATCH_STRIDE    = 200      # non-overlap to match pretrain (S = T//P)
MIN_PAD         = 0        # pad right if not divisible (set 0 for T=1000, P=200)

# Train config
BATCH_SIZE      = 4
EPOCHS          = 25
LR              = 1e-5     # 작은 LR 권장 (transformer 기반 파인튜닝 안정화)
N_FOLDS         = 5
NUM_WORKERS     = 4

# Pretrained checkpoint (로컬에 이미 있다고 했음)
PRETRAINED_CKPT = "/home/RA/EEG_Challenge/Challenge2/CBraMod/pretrained_weights/pretrained_weights.pth"  # e.g. ./pretrained_weights/pretrained_weights.pth
# (대안) HF에서 받기: https://huggingface.co/weighting666/CBraMod  (수동 다운로드)  # :contentReference[oaicite:5]{index=5}

SEED = 42
DEBUG = True
torch.manual_seed(SEED)
np.random.seed(SEED)

print("Device:", DEVICE)

# ------------------------------------------------------------
# 1) participants.tsv 병합 및 p_factor 로드
# ------------------------------------------------------------
def load_all_participants(bids_root, datasets):
    dfs = []
    for ds in datasets:
        pfile = os.path.join(bids_root, ds, "participants.tsv")
        if os.path.exists(pfile):
            df = pd.read_csv(pfile, sep="\t")
            df["release"] = ds
            dfs.append(df)
    df_all = pd.concat(dfs, ignore_index=True)
    df_all = df_all.drop_duplicates(subset=["participant_id"])
    df_all = df_all.set_index("participant_id")
    return df_all

meta_df   = load_all_participants(BIDS_ROOT, DATASETS)
labels_df = meta_df[["p_factor"]].dropna().copy()
labels_df = labels_df[labels_df["p_factor"].apply(np.isfinite)]

# (선택) 라벨 z-score → 발산 방지/수렴 안정화에 유리
USE_LABEL_ZSCORE = True
if USE_LABEL_ZSCORE:
    mu = labels_df["p_factor"].mean()
    sd = labels_df["p_factor"].std() + 1e-6
    labels_df["p_factor_norm"] = (labels_df["p_factor"] - mu) / sd
    TARGET_COL = "p_factor_norm"
else:
    TARGET_COL = "p_factor"

# ------------------------------------------------------------
# 2) 캐시 파일 목록
# ------------------------------------------------------------
all_files = [f for f in os.listdir(CACHE_DIR) if f.endswith(".npy")]
train_pool_files = [f for f in all_files if "_train_" in f]
ext_val_files    = [f for f in all_files if "_val_"   in f]  # release5 holdout

def subj_from_fname(fname): return fname.split("_")[0]

# ------------------------------------------------------------
# 3) StratifiedGroupKFold (p-factor 분위 bin)
# ------------------------------------------------------------
rows = []
for f in train_pool_files:
    subj = subj_from_fname(f)
    if subj in labels_df.index:
        rows.append({"fname": f, "subj": subj, "p": float(labels_df.loc[subj, TARGET_COL])})
df = pd.DataFrame(rows)
df["p_bin"] = pd.qcut(df["p"], q=5, labels=False, duplicates="drop")

cv = StratifiedGroupKFold(n_splits=N_FOLDS, shuffle=True, random_state=SEED)
folds = list(cv.split(df, y=df["p_bin"], groups=df["subj"]))
print("Bin counts:\n", df["p_bin"].value_counts().sort_index())

# ------------------------------------------------------------
# 4) 유틸: (C, T) → (C, S, P) 패치 변환
# ------------------------------------------------------------
def to_patches(x_ct, patch_size=PATCH_SIZE, stride=PATCH_STRIDE, min_pad=MIN_PAD):
    """
    x_ct: (C, T) numpy array
    returns: (C, S, P) numpy array
    """
    C, T = x_ct.shape
    # pad to multiple of patch_size if needed
    if T < patch_size:
        pad = patch_size - T
        x_ct = np.pad(x_ct, ((0,0),(0,pad)), mode="constant")
        T = x_ct.shape[1]
    if (T - patch_size) % stride != 0:
        # right pad so that (T - patch_size) is divisible by stride
        pad_needed = (stride - ((T - patch_size) % stride)) % stride
        if pad_needed > 0:
            x_ct = np.pad(x_ct, ((0,0),(0,pad_needed)), mode="constant")
            T = x_ct.shape[1]

    S = 1 + (T - patch_size) // stride
    patches = np.zeros((C, S, patch_size), dtype=x_ct.dtype)
    for s in range(S):
        start = s * stride
        end = start + patch_size
        patches[:, s, :] = x_ct[:, start:end]
    return patches  # (C, S, P)

# ------------------------------------------------------------
# 5) Dataset: 캐시 → 정규화 → 패치화
# ------------------------------------------------------------
class EEGCachedDatasetCBraMod(Dataset):
    def __init__(self, cache_dir, rows, labels_df, n_segments_per_item=4, train=True):
        self.cache_dir = cache_dir
        self.rows = rows.reset_index(drop=True)
        self.labels_df = labels_df
        self.n_segments_per_item = n_segments_per_item
        self.train = train

    def __len__(self): return len(self.rows)

    def __getitem__(self, idx):
        r = self.rows.iloc[idx]
        path = os.path.join(self.cache_dir, r.fname)
        X_all = np.load(path, allow_pickle=True)  # (Sg, C, T) or (Sg, T, C)
        X_all = np.nan_to_num(X_all, nan=0.0, posinf=0.0, neginf=0.0)

        # pick some segments → crop/avg or just mean over segments for subject-level stability
        Sg = X_all.shape[0]
        k = min(self.n_segments_per_item, Sg)
        sel = np.random.choice(Sg, size=k, replace=(Sg < k)) if self.train else np.arange(Sg)
        X = X_all[sel].mean(axis=0)  # (C,T) 혹은 (T,C)

        # axes fix
        if X.shape[0] != N_CHANS and X.shape[1] == N_CHANS:
            X = X.T
        # channel align
        if X.shape[0] > N_CHANS:
            X = X[:N_CHANS, :]
        elif X.shape[0] < N_CHANS:
            X = np.pad(X, ((0, N_CHANS - X.shape[0]), (0, 0)), mode="constant")

        # z-score + gentle clip (과도한 clip은 정보 손실)
        mean, std = np.mean(X), np.std(X)
        std = 1e-6 if (not np.isfinite(std) or std < 1e-6) else std
        X = (X - mean) / std
        X = np.clip(X, -6.0, 6.0)

        # → (C, S, P) 패치
        X_patches = to_patches(X, patch_size=PATCH_SIZE, stride=PATCH_STRIDE, min_pad=MIN_PAD)

        X = torch.tensor(X_patches, dtype=torch.float32).contiguous()  # (C,S,P)
        if self.train:
            X = X + 1e-5 * torch.randn_like(X)  # micro-noise for stability

        y = torch.tensor(float(self.labels_df.loc[r.subj, TARGET_COL]), dtype=torch.float32)
        return X, y

# ------------------------------------------------------------
# 6) 모델: CBraMod + 회귀 헤드
#    (README Quick Start: model = CBraMod(); load_state_dict(...); model.proj_out = Identity)
#    여기서는 C,S 평균 풀링 → 200-d → MLP(200→128→1)
# ------------------------------------------------------------
class CBraModRegressor(nn.Module):
    def __init__(self, n_chans=N_CHANS, patch_size=PATCH_SIZE, sfreq=SFREQ, pretrained_ckpt=PRETRAINED_CKPT, freeze_backbone=False):
        super().__init__()
        # 1) 백본
        self.backbone = CBraMod()  # 내부 기본 설정 사용 (사전학습과 호환)  # :contentReference[oaicite:6]{index=6}

        # 2) 가중치 로드 (pretrained)
        state = torch.load(pretrained_ckpt, map_location="cpu")
        self.backbone.load_state_dict(state, strict=True)  # README 예시대로 로드  # :contentReference[oaicite:7]{index=7}

        # 3) 출력 프로젝션 제거 (원 논문/README 예시와 동일)
        self.backbone.proj_out = nn.Identity()            # feature map 그대로 꺼내 쓸 것  # :contentReference[oaicite:8]{index=8}

        if freeze_backbone:
            for p in self.backbone.parameters():
                p.requires_grad = False

        # 백본 출력 차원 추정:
        # README Quick Start의 분류 예시에서 입력 (B,22,4,200) → 출력을 'b c s p'로 펼쳐 22*4*200을 Linear에 투입
        # 즉, 마지막 축이 200-d feature로 해석 가능 (patch 임베딩 200)  # :contentReference[oaicite:9]{index=9}
        self.embed_dim = 200

        # 평균 풀링 (C,S) → 200
        self.pool = lambda z: z.mean(dim=(1,2))  # (B,C,S,200) → (B,200)

        # 회귀 헤드
        self.reg_head = nn.Sequential(
            nn.LayerNorm(self.embed_dim, eps=1e-3),
            nn.Linear(self.embed_dim, 128),
            nn.ReLU(inplace=True),
            nn.Dropout(0.2),
            nn.Linear(128, 1)
        )

    def forward(self, x_csp):   # x: (B,C,S,P)
        z = self.backbone(x_csp)  # 예상 (B, C, S, 200)  (README 플로우)  # :contentReference[oaicite:10]{index=10}
        if z.ndim != 4 or z.shape[-1] != self.embed_dim:
            # 안전장치: 혹시 구조가 다르면 평균축 조정
            z = z.view(z.shape[0], -1, self.embed_dim)  # (B, C*S, 200) 가정
            z = z.mean(dim=1)                           # (B,200)
            return self.reg_head(z).squeeze(-1)

        z = self.pool(z)               # (B,200)
        out = self.reg_head(z).squeeze(-1)  # (B,)
        return out

# ------------------------------------------------------------
# 7) 학습 루프 (한 fold)
# ------------------------------------------------------------
def train_one_fold(train_rows, valid_rows, fold_id):
    train_ds = EEGCachedDatasetCBraMod(CACHE_DIR, train_rows, labels_df, train=True)
    valid_ds = EEGCachedDatasetCBraMod(CACHE_DIR, valid_rows, labels_df, train=False)

    train_loader = DataLoader(train_ds, batch_size=BATCH_SIZE, shuffle=True,
                              num_workers=NUM_WORKERS, pin_memory=True)
    valid_loader = DataLoader(valid_ds, batch_size=BATCH_SIZE, shuffle=False,
                              num_workers=NUM_WORKERS, pin_memory=True)

    model = CBraModRegressor(pretrained_ckpt=PRETRAINED_CKPT, freeze_backbone=False).to(DEVICE)
    opt = torch.optim.AdamW(model.parameters(), lr=LR, weight_decay=1e-3)

    best = {"mae": np.inf, "mse": np.inf, "r2": -1}

    for ep in range(1, EPOCHS + 1):
        # ---------------------- TRAIN ----------------------
        model.train()
        train_preds, train_trues = [], []
        train_loss = 0.0

        for X, y in tqdm(train_loader, desc=f"[Fold {fold_id}] Epoch {ep} (train)", leave=False):
            X, y = X.to(DEVICE), y.to(DEVICE)
            pred = model(X)
            if torch.isnan(pred).any() or torch.isinf(pred).any():
                if DEBUG: print("[Train] NaN/Inf pred → skip")
                continue

            loss = 0.8 * F.l1_loss(pred, y) + 0.2 * F.mse_loss(pred, y)
            if torch.isnan(loss) or torch.isinf(loss):
                continue

            opt.zero_grad()
            loss.backward()
            torch.nn.utils.clip_grad_norm_(model.parameters(), 0.5)
            opt.step()

            train_loss += loss.item()
            train_preds.extend(pred.detach().cpu().numpy())
            train_trues.extend(y.cpu().numpy())

        # --- train metrics ---
        train_preds = np.array(train_preds, dtype=np.float32)
        train_trues = np.array(train_trues, dtype=np.float32)
        mask = np.isfinite(train_preds) & np.isfinite(train_trues)
        train_preds, train_trues = train_preds[mask], train_trues[mask]
        if len(train_trues) > 0:
            train_mae = mean_absolute_error(train_trues, train_preds)
            train_mse = mean_squared_error(train_trues, train_preds)
            train_r2  = r2_score(train_trues, train_preds)
        else:
            train_mae = train_mse = train_r2 = np.nan

        # ---------------------- VALID ----------------------
        model.eval()
        val_preds, val_trues = [], []

        with torch.no_grad():
            for X, y in tqdm(valid_loader, desc=f"[Fold {fold_id}] Epoch {ep} (val)", leave=False):
                X, y = X.to(DEVICE), y.to(DEVICE)
                p = model(X)
                val_preds.extend(p.cpu().numpy())
                val_trues.extend(y.cpu().numpy())

        val_preds = np.array(val_preds, dtype=np.float32)
        val_trues = np.array(val_trues, dtype=np.float32)
        mask = np.isfinite(val_preds) & np.isfinite(val_trues)
        val_preds, val_trues = val_preds[mask], val_trues[mask]
        if len(val_trues) > 0:
            val_mae = mean_absolute_error(val_trues, val_preds)
            val_mse = mean_squared_error(val_trues, val_preds)
            val_r2  = r2_score(val_trues, val_preds)
        else:
            val_mae = val_mse = val_r2 = np.nan

        # ---------------------- LOG ----------------------
        print(f"[Fold {fold_id}] Ep{ep:02d} | "
              f"Train: MAE={train_mae:.4f}, MSE={train_mse:.4f}, R²={train_r2:.3f} | "
              f"Val: MAE={val_mae:.4f}, MSE={val_mse:.4f}, R²={val_r2:.3f}")

        # ---------------------- SAVE ----------------------
        if np.isfinite(val_mae) and val_mae < best["mae"]:
            best = {"mae": val_mae, "mse": val_mse, "r2": val_r2}
            torch.save(model.state_dict(), f"cbramod_fold{fold_id}.pth")

    return best

# ------------------------------------------------------------
# 8) Cross-validation 실행
# ------------------------------------------------------------
cv_results = []
for fold_id, (tr_idx, va_idx) in enumerate(folds, 1):
    train_rows = df.iloc[tr_idx]
    valid_rows = df.iloc[va_idx]
    best = train_one_fold(train_rows, valid_rows, fold_id)
    cv_results.append(best)

mean_mae = np.nanmean([r["mae"] for r in cv_results])
mean_mse = np.nanmean([r["mse"] for r in cv_results])
mean_r2  = np.nanmean([r["r2"] for r in cv_results])

print("\n=== StratifiedGroupKFold CV Results ===")
print(f"MAE={mean_mae:.4f} | MSE={mean_mse:.4f} | R²={mean_r2:.4f}")

# ------------------------------------------------------------
# 9) (옵션) Release 5 Hold-out 검증
# ------------------------------------------------------------
if len(ext_val_files) > 0:
    ext_rows = []
    for f in ext_val_files:
        subj = subj_from_fname(f)
        if subj in labels_df.index:
            ext_rows.append({"fname": f, "subj": subj})
    ext_df = pd.DataFrame(ext_rows)

    ext_ds = EEGCachedDatasetCBraMod(CACHE_DIR, ext_df, labels_df, train=False)
    ext_loader = DataLoader(ext_ds, batch_size=BATCH_SIZE, shuffle=False, num_workers=NUM_WORKERS, pin_memory=True)

    model = CBraModRegressor(pretrained_ckpt=PRETRAINED_CKPT, freeze_backbone=False).to(DEVICE)
    state = torch.load("cbramod_fold1.pth", map_location=DEVICE)
    model.load_state_dict(state, strict=False)
    model.eval()

    preds, trues = [], []
    with torch.no_grad():
        for X, y in tqdm(ext_loader, desc="External Val (Release5)"):
            X, y = X.to(DEVICE), y.to(DEVICE)
            p = model(X)
            preds.extend(p.cpu().numpy())
            trues.extend(y.cpu().numpy())

    preds, trues = np.array(preds, np.float32), np.array(trues, np.float32)
    mask = np.isfinite(preds) & np.isfinite(trues)
    preds, trues = preds[mask], trues[mask]

    val_mae = mean_absolute_error(trues, preds)
    val_mse = mean_squared_error(trues, preds)
    val_r2  = r2_score(trues, preds)
    print(f"\n[Release5 Hold-out] MAE={val_mae:.4f} | MSE={val_mse:.4f} | R²={val_r2:.4f}")


Device: cuda
Bin counts:
 p_bin
0    62
1    62
2    61
3    62
4    62
Name: count, dtype: int64


[Fold 1] Ep01 | Train: MAE=0.8211, MSE=1.0641, R²=-0.107 | Val: MAE=0.8542, MSE=1.2080, R²=-0.016


[Fold 1] Ep02 | Train: MAE=0.7935, MSE=0.9990, R²=-0.039 | Val: MAE=0.8521, MSE=1.1954, R²=-0.005


[Fold 1] Ep03 | Train: MAE=0.7920, MSE=0.9855, R²=-0.025 | Val: MAE=0.8560, MSE=1.2260, R²=-0.031


[Fold 1] Ep04 | Train: MAE=0.7862, MSE=0.9912, R²=-0.031 | Val: MAE=0.8570, MSE=1.1870, R²=0.002


...

[Fold 5] Ep23 | Train: MAE=0.7932, MSE=1.0228, R²=-0.012 | Val: MAE=0.8884, MSE=1.2248, R²=-0.280


[Fold 5] Ep24 | Train: MAE=0.7965, MSE=1.0287, R²=-0.018 | Val: MAE=0.8417, MSE=1.1085, R²=-0.158


[Fold 5] Ep25 | Train: MAE=0.7932, MSE=1.0299, R²=-0.019 | Val: MAE=0.8648, MSE=1.1643, R²=-0.216

=== StratifiedGroupKFold CV Results ===
MAE=0.7889 | MSE=1.0002 | R²=-0.0134


External Val (Release5): 100%|██████████| 2/2 [00:04<00:00,  2.46s/it]


[Release5 Hold-out] MAE=0.9827 | MSE=1.5127 | R²=-0.0078


In [25]:
import numpy as np

path = "/data5/open_data/HBN/cache_eeg_windows/sub-NDARBK082PDD_task-contrastChangeDetection_run-2_eeg_train_100Hz.npy"


data = np.load(path, allow_pickle=True)


print("Shape:", data.shape)
print("Type:", type(data))
print("First 5 value:\n", data[:5])

Shape: (179, 129, 1000)
Type: <class 'numpy.ndarray'>
First 5 value:
 [[[-8.63939285e-01  1.61699219e+01  7.41871262e+00 ... -8.69628811e+00
   -1.43299551e+01 -1.53498878e+01]
  [-5.82120240e-01  9.46152115e+00 -5.84083617e-01 ...  8.31683064e+00
    3.66228795e+00  2.41324067e+00]
  [-4.50548977e-01  6.11197758e+00 -3.43022561e+00 ...  1.21772289e+01
    1.02824030e+01  1.15594759e+01]
  ...
  [ 3.13054638e+01 -5.44049255e+02 -2.99364502e+02 ...  1.80678055e+02
    1.92680817e+02  1.76027405e+02]
  [-7.58383721e-02 -7.36798286e+00 -2.32637157e+01 ... -3.39655342e+01
   -3.57121658e+01 -3.75012512e+01]
  [ 0.00000000e+00  0.00000000e+00  0.00000000e+00 ...  0.00000000e+00
    0.00000000e+00  0.00000000e+00]]

 [[ 1.47397890e+01  9.74559021e+00  1.66246052e+01 ...  4.34740257e+01
    4.19287186e+01  4.31210175e+01]
  [ 1.95440331e+01  1.90705624e+01  2.45238895e+01 ...  5.06877861e+01
    4.21373940e+01  4.38972473e+01]
  [ 9.85815144e+00  7.06270695e+00  1.01744957e+01 ...  2.87864208